In [1]:
#import os
#os.environ["CUDA_VISIBLE_DEVICES"]

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/tmp/ipykernel_505606/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
#!gdown "1iUNtsGGS2rFgvtGy9wp8H1mnfFh0OmuZ&confirm=t"#PascalTrainVal_VicgabusaAzure#20230610
#!gdown "15UYA8GQ7D-bS1bxY1_t51GjsUL8yfwo1&confirm=t"#Embryo_VicgabusaAzure#20230610

#!unzip PascalTrainVal.zip
#!unzip embryoDataset.zip

# CLASE DEEPLAB

In [4]:
import os
import cv2
import numpy as np
from glob import glob
from scipy.io import loadmat
import matplotlib.pyplot as plt

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

2024-02-08 18:34:29.104167: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-08 18:34:29.676141: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/vbugueno/.local/lib/python3.9/site-packages/cv2/../../lib64:
2024-02-08 18:34:29.676185: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/vbugueno/.local/lib/python3.9/site-packages/cv2/../../lib

In [5]:
#lst_imagenes[0]

In [6]:
#auxim=cv2.imread(lst_imagenes[1])
#print(auxim.shape)
#print(np.unique(auxim))
#print(type(auxim))

In [7]:
#DATA_DIR = "./COLAB_instance-level-human-parsing2/embrion/secuencia"
#sorted(glob(os.path.join(DATA_DIR, "raw/*")))[1:1+10]

In [8]:
#def myfunc(a,b, *args, **kwargs):
#    for ar in args:
#        print (ar)
#myfunc(1,2,3,4,5,6)

In [9]:
if False:
    listaFiles=sorted(glob(r"./"+r"COLAB_instance-level-human-parsing2/embryoDataset"+r"/"+r"video_animal2_proyeccion_SeqDA/*"))
    #listaFiles=[x.split(r'')[0] for x in listaFiles]
    listaFiles

    import re

    out=[]
    for listaFiles_i in listaFiles:
        aux=re.search(r"[^/]+$", listaFiles_i)[0]
        aux=re.match(r".*(?=\.)",aux)[0]
        out.append(aux)

    out

In [10]:
if False:
    def has_common_string(list1, list2, list3):
        set1 = set(list1)
        set2 = set(list2)
        set3 = set(list3)
        common_strings = set1.intersection(set2)
        common_strings = common_strings.intersection(set3)
        print("common_strings: ", common_strings)
        return len(common_strings) > 0


    list1 = ['kiwi', 'banana', 'orange']
    list2 = ['banana', 'grape', 'kiwi']
    list3 = ['orange', 'kiwi', 'watermelon']



    if has_common_string(list1, list2, list3):
        print("The three lists have a common string.")
    else:
        print("The three lists do not have a common string.")


In [11]:
class class_DeepLab():
    
    def __call__(self, 
                 str_entrenaroCargarIn, 
                 str_dirModelLoadIn, 
                 boo_guardarModeloIn, 
                 int_image_sizeIn, 
                 int_batch_sizeIn, 
                 int_num_classesIn, 
                 str_data_dirIn, 
                 str_mask_folderIn,
                 str_raw_folderIn, 
                 int_num_train_imagesIn, 
                 int_num_val_imagesIn, 
                 int_num_test_imagesIn, 
                 int_epocasIn,
                 boo_activarCRFIn
                ):
        """entrenaroCargar tiene 2 opciones: Train o Load """
        
        self.str_entrenaroCargar=str_entrenaroCargarIn
        self.str_dirModelLoad=str_dirModelLoadIn
        self.boo_guardarModeloIn=boo_guardarModeloIn
        
        self.int_IMAGE_SIZE = int_image_sizeIn#Se usa en mode, en deeplabv3+ y read_image
        self.int_BATCH_SIZE = int_batch_sizeIn
        self.int_NUM_CLASSES = int_num_classesIn
        #DATA_DIR = "./instance-level_human_parsing/instance-level_human_parsing/Training"
        #DATA_DIR = "./COLAB_instance-level-human-parsing2/instance-level_human_parsing/instance-level_human_parsing/Training"
        str_DATA_DIR = r"./"+str_data_dirIn
        str_MASK_Folder=str_mask_folderIn+r"/*" #EVLstack_SeqDA
        str_RAW_Folder=str_raw_folderIn+r"/*" #video_animal2_proyeccion_SeqDA2
        int_NUM_TRAIN_IMAGES = int_num_train_imagesIn
        int_NUM_VAL_IMAGES = int_num_val_imagesIn
        #NUM_TEST_IMAGES = 4000
        int_NUM_TEST_IMAGES = int_num_test_imagesIn
        int_EPOCAS=int_epocasIn
        
        from datetime import datetime as dt  #20230119_1734
        str_fechaInicio=dt.today().strftime('%Y%m%d_%H%M')#20230119_1734
            

            
        
        if(self.str_entrenaroCargar!="Train" and self.str_entrenaroCargar!="Load"):#20230619
            raise Exception("Escojer Train o Load o no tiene sentido correr esta clase")  
            
        ####
        print("deeplab str_data_dirIn: ", str_data_dirIn)
        
        if(str_data_dirIn=="PascalTrainVal"):
            str_DatasetNombre = "DatasetPascal"
        elif(str_data_dirIn=="COLAB_instance-level-human-parsing2/embryoDataset"):
            str_DatasetNombre = "DatasetEmbrion"
        elif(str_data_dirIn=="embryoDataset_SinDA"):
            str_DatasetNombre = "DatasetEmbrionSinDA"
        elif(str_data_dirIn==r"2D_Neuronal_dataset_Proc"):
            str_DatasetNombre = "DatasetNeuron"
        elif(str_data_dirIn==r"Warwick_QU_Dataset_Released 2016_07_08_Proc"):
            str_DatasetNombre = "DatasetTissue"
        else:
            raise Exception("Si str_data_dirIn no es PascalTrainVal ni tampoco COLAB_instance-level-human-parsing2/embryoDataset ni neuron. Tengo que saber si es de pascal para poenrle ignore label en el if")    
        ###
        ###
        if(self.str_entrenaroCargar=="Load"):
            if((str_DatasetNombre=="DatasetPascal") and 
               ("pascal" not in self.str_dirModelLoad.lower())
              ):
                raise Exception("Ojo parece que se esta cargando un modelo que no se condice con el dataset entrenado, en el nombre deberia salir pascal. El nombre del modelo entrenado no contiene la palabra de del dataset extraida de str_data_dirIn")    
            if((str_DatasetNombre=="DatasetEmbrion") and 
               (("embryo" not in self.str_dirModelLoad.lower()) and
                ("embrion" not in self.str_dirModelLoad.lower())
               )
              ):
                raise Exception("Ojo parece que se esta cargando un modelo que no se condice con el dataset entrenado, en el nombre deberia salir embryo. El nombre del modelo entrenado no contiene la palabra de del dataset extraida de str_data_dirIn")    
            if((str_DatasetNombre=="embryoDataset_SinDA") and 
               ("embryo" not in self.str_dirModelLoad.lower())
              ):
                raise Exception("Ojo parece que se esta cargando un modelo que no se condice con el dataset entrenado, en el nombre deberia salir embryo. El nombre del modelo entrenado no contiene la palabra de del dataset extraida de str_data_dirIn")    
            
        
        if (str_DatasetNombre == "DatasetPascal"):
            
            ####
            lst_train_masks = sorted(glob(os.path.join(str_DATA_DIR, str_MASK_Folder)))[1:1+int_NUM_TRAIN_IMAGES]
            lst_train_raws = sorted(glob(os.path.join(str_DATA_DIR, str_RAW_Folder)))[1:1+int_NUM_TRAIN_IMAGES]#20230123_1544
            
            ####
            lst_val_masks = sorted(glob(os.path.join(str_DATA_DIR, str_MASK_Folder)))[
              1+int_NUM_TRAIN_IMAGES : 1+ int_NUM_TRAIN_IMAGES+ int_NUM_VAL_IMAGES
            ]
            lst_val_raws = sorted(glob(os.path.join(str_DATA_DIR, str_RAW_Folder)))[#20230123_1544
              1+int_NUM_TRAIN_IMAGES : 1+ int_NUM_TRAIN_IMAGES+ int_NUM_VAL_IMAGES#20230123_1544
            ]#20230123_1544
            
            ####
            lst_test_masks = sorted(glob(os.path.join(str_DATA_DIR, str_MASK_Folder)))[
              1+ int_NUM_TRAIN_IMAGES+ int_NUM_VAL_IMAGES : 1+ int_NUM_TRAIN_IMAGES+ int_NUM_VAL_IMAGES + int_NUM_TEST_IMAGES
            ]
            lst_test_raws = sorted(glob(os.path.join(str_DATA_DIR, str_RAW_Folder)))[#20230123_1544
              1+ int_NUM_TRAIN_IMAGES+ int_NUM_VAL_IMAGES : 1+ int_NUM_TRAIN_IMAGES+ int_NUM_VAL_IMAGES + int_NUM_TEST_IMAGES#20230123_1544
            ]#20230123_1544
        
            lst_train_images, lst_val_images, lst_test_images = lst_train_raws, lst_val_raws, lst_test_raws

        
        
        if(str_DatasetNombre == "DatasetEmbrion"):#Esto es para sacar a los data augmented de test y validation
            if (int_num_train_imagesIn+int_num_val_imagesIn+int_num_test_imagesIn>145):
                raise Exception("Hay solo 145 imagenes, hay que achicar int_testFilescount o int_validationFilescount o int_trainingFilescount")
                
        if(str_DatasetNombre == "DatasetNeuron"):#Esto es para sacar a los data augmented de test y validation
            if (int_num_train_imagesIn+int_num_val_imagesIn+int_num_test_imagesIn>210):
                raise Exception("Hay solo 210 imagenes, hay que achicar int_testFilescount o int_validationFilescount o int_trainingFilescount")
        
        if(str_DatasetNombre == "DatasetTissue"):#Esto es para sacar a los data augmented de test y validation
            if (int_num_train_imagesIn+int_num_val_imagesIn+int_num_test_imagesIn>165):
                raise Exception("Hay solo 165 imagenes, hay que achicar int_testFilescount o int_validationFilescount o int_trainingFilescount")
        
        
        if(str_DatasetNombre == "DatasetEmbrion" or str_DatasetNombre == "DatasetNeuron" or str_DatasetNombre == "DatasetTissue"):#Esto es para sacar a los data augmented de test y validation
            #str_mask_folder = r"COLAB_instance-level-human-parsing2/embryoDataset/EVLstack_SeqDA"
            #str_raw_folder = r"COLAB_instance-level-human-parsing2/embryoDataset/video_animal2_proyeccion_SeqDA2"
            #str_mask_folder = r"COLAB_instance-level-human-parsing2/embryoDataset/EVLstack_SeqDA"
            #str_raw_folder = r"COLAB_instance-level-human-parsing2/embryoDataset/video_animal2_proyeccion_SeqDA2"
            str_mask_folder= os.path.join(str_DATA_DIR, str_mask_folderIn)
            str_raw_folder= os.path.join(str_DATA_DIR, str_raw_folderIn)
            
            #print("str_mask_folder: ", str_mask_folder)
            #print("str_raw_folder: ", str_raw_folder)

            
            


            #seed = 123 
            #import random
            #random.seed(seed)
            
            
            lst_str_filenamePath_10R_Masks = glob(os.path.join(str_mask_folder, "1OR*"))#1 (para que quede al principio) y OR de original
            lst_str_filenamePath_10R_Raws = glob(os.path.join(str_raw_folder, "1OR*"))#Parece que no se usa

            lst_str_filenamePath_Masks = glob(str_mask_folder + '/*')
            lst_str_filenamePath_Raws = glob(str_raw_folder + '/*')
            
            
            #print(r"lst_str_filenamePath_10R_Masks: ", os.path.join(str_mask_folder, "1OR*"))
            #print(r"lst_str_filenamePath_Masks: ", str_mask_folder + '/*')
            
            #for lst_str_filenamePath_Masks_i in lst_str_filenamePath_Masks:
            #    print("lst_str_filenamePath_Masks_i: ", lst_str_filenamePath_Masks_i)
            
            #for lst_str_filenamePath_10R_Masks_i in lst_str_filenamePath_10R_Masks:
            #    print("lst_str_filenamePath_10R_Masks_i: ", lst_str_filenamePath_10R_Masks_i)
            
            
            
            
            #Test
            #import random
            #random.seed(11)
            #np.random.seed(11)
            #gen = np.random.default_rng()
            
            #lst_str_filenamePath_10R_testSet = random.sample(set(lst_str_filenamePath_10R_Masks), int_num_test_imagesIn)#Selecciono solo de entre 1OR que es la imagen sin ninguna rotacion
            #int_seed=1
            #np.random.seed(int_seed)
            
            import random

            #lower = 1
            #upper = 10
            int_seed = random.randint(1, 10000)#Redflag. QUEDA DETERMINISTICO SI ESTO SE DEJA FIJO
            int_seed = 3873#Redflag. QUEDA DETERMINISTICO SI ESTO SE DEJA FIJO
            np.random.seed(int_seed)
            #print(random_int)
            
            
            
            if False:
                my_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
                sample = np.random.choice(my_list, size=3, replace=False)
                print("sample: ", sample)
                my_list = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20"]
                sample = np.random.choice(my_list, size=3, replace=False)
                print("sample: ", sample)
                #print("list(set(lst_str_filenamePath_10R_Masks))", list(set(lst_str_filenamePath_10R_Masks)))
                print("int_num_test_imagesIn", int_num_test_imagesIn)

            
            #Test
            lst_str_filenamePath_10R_testSet = np.random.choice(lst_str_filenamePath_10R_Masks, int_num_test_imagesIn, replace=False)#Selecciono solo de entre 1OR que es la imagen sin ninguna rotacion
            lst_str_filenamePath_10R_testSet = sorted(lst_str_filenamePath_10R_testSet)
            
            
            #np.random.choice(my_list, size=3, replace=False)
            #lst_str_filenamePath_10R_testSet = random.sample(set(lst_str_filenamePath_10R_Masks), int_num_test_imagesIn)#Selecciono solo de entre 1OR que es la imagen sin ninguna rotacion
            #lst_str_filenamePath_10R_testSet = gen.choice(list(set(lst_str_filenamePath_10R_Masks)), int_num_test_imagesIn, replace=False)
            
            #my_array = np.array(set(lst_str_filenamePath_10R_Masks))
            #lst_str_filenamePath_10R_testSet = np.random.choice(my_array, int_num_test_imagesIn, replace=False).tolist()#Selecciono solo de entre 1OR que es la imagen sin ninguna rotacion
            #lst_str_filename_10R_testSet = [os.path.splitext(os.path.basename(path))[0] for path in lst_str_filenamePath_10R_testSet]#Me quedo con los filenames. Ejemplo: 1OR0027
            #lst_str_filenamePath_10R_testSet_Masks = [path for path in lst_str_filenamePath_Masks if os.path.splitext(os.path.basename(path))[0] in lst_str_filename_10R_testSet]#Recupero el path a partir de los filenames para Mask
            #lst_str_filenamePath_10R_testSet_Raws = [path for path in lst_str_filenamePath_Raws if os.path.splitext(os.path.basename(path))[0] in lst_str_filename_10R_testSet]#Recupero el path a partir de los filenames para Raws
            lst_str_filename_10R_testSet = [os.path.splitext(file)[0][-4:]  for file in lst_str_filenamePath_10R_testSet] # Me quedo solo con el nombre del archivo sin prefijos ni sufijos y con extensión
            lst_str_filenamePath_testSet_Masks = [file_path for file_path in lst_str_filenamePath_Masks if any(substr in file_path for substr in lst_str_filename_10R_testSet)]
            lst_str_filenamePath_testSet_Raws = [file_path for file_path in lst_str_filenamePath_Raws if any(substr in file_path for substr in lst_str_filename_10R_testSet)]

            
            #Validation
            set_aux_validationSet= [x for x in lst_str_filenamePath_10R_Masks if x not in lst_str_filenamePath_10R_testSet]
            #lst_str_filenamePath_10R_validationSet = random.sample(set(lst_str_filenamePath_10R_Masks)-set(lst_str_filenamePath_10R_testSet), int_num_val_imagesIn)#Selecciono solo de entre 1OR que es la imagen sin ninguna rotacion
            lst_str_filenamePath_10R_validationSet = np.random.choice(set_aux_validationSet, int_num_val_imagesIn, replace=False)#Selecciono solo de entre 1OR que es la imagen sin ninguna rotacion
            lst_str_filenamePath_10R_validationSet = sorted(lst_str_filenamePath_10R_validationSet)

            
            #my_array = np.array(set(lst_str_filenamePath_10R_Masks)-set(lst_str_filenamePath_10R_testSet))
            #lst_str_filenamePath_10R_validationSet = np.random.choice(set(lst_str_filenamePath_10R_Masks)-set(lst_str_filenamePath_10R_testSet), int_num_val_imagesIn, replace=False)#Selecciono solo de entre 1OR que es la imagen sin ninguna rotacion
            #lst_str_filename_10R_validationSet = [os.path.splitext(os.path.basename(path))[0] for path in lst_str_filenamePath_10R_validationSet]#Me quedo con los filenames    
            #lst_str_filenamePath_10R_validationSet_Masks = [path for path in lst_str_filenamePath_Masks if os.path.splitext(os.path.basename(path))[0] in lst_str_filename_10R_validationSet]
            #lst_str_filenamePath_10R_validationSet_Raws = [path for path in lst_str_filenamePath_Raws if os.path.splitext(os.path.basename(path))[0] in lst_str_filename_10R_validationSet]
            lst_str_filename_10R_validationSet = [os.path.splitext(file)[0][-4:]  for file in lst_str_filenamePath_10R_validationSet] # Me quedo solo con el nombre del archivo sin prefijos ni sufijos y con extensión
            lst_str_filenamePath_validationSet_Masks = [file_path for file_path in lst_str_filenamePath_Masks if any(substr in file_path for substr in lst_str_filename_10R_validationSet)]
            lst_str_filenamePath_validationSet_Raws = [file_path for file_path in lst_str_filenamePath_Raws if any(substr in file_path for substr in lst_str_filename_10R_validationSet)]

            
            #Training
            set_aux_trainingSet= [x for x in lst_str_filenamePath_10R_Masks if x not in set_aux_validationSet]
            #lst_str_filenamePath_10R_trainingSet = random.sample(set(lst_str_filenamePath_10R_Masks)-set(lst_str_filenamePath_10R_testSet)-set(lst_str_filenamePath_10R_validationSet), int_num_train_imagesIn) #Selecciono solo de entre 1OR que es la imagen sin ninguna rotacion
            lst_str_filenamePath_10R_trainingSet = np.random.choice(list(set(lst_str_filenamePath_10R_Masks)-set(lst_str_filenamePath_10R_testSet)-set(lst_str_filenamePath_10R_validationSet)), int_num_train_imagesIn, replace=False) #Selecciono solo de entre 1OR que es la imagen sin ninguna rotacion
            lst_str_filenamePath_10R_trainingSet = sorted(lst_str_filenamePath_10R_trainingSet)

            
            #lst_str_filenamePath_10R_trainingSet = np.random.choice(set(lst_str_filenamePath_10R_Masks)-set(lst_str_filenamePath_10R_testSet)-set(lst_str_filenamePath_10R_validationSet), int_num_train_imagesIn, replace=False) #Selecciono solo de entre 1OR que es la imagen sin ninguna rotacion
            lst_str_filename_10R_trainingSet = [os.path.splitext(file)[0][-4:]  for file in lst_str_filenamePath_10R_trainingSet] # Me quedo solo con el nombre del archivo sin prefijos ni sufijos y con extensión
            lst_str_filenamePath_trainingSet_Masks = [file_path for file_path in lst_str_filenamePath_Masks if any(substr in file_path for substr in lst_str_filename_10R_trainingSet)]
            lst_str_filenamePath_trainingSet_Raws = [file_path for file_path in lst_str_filenamePath_Raws if any(substr in file_path for substr in lst_str_filename_10R_trainingSet)]

            if True:
                print("lst_str_filenamePath_10R_testSet: ")
                for lst_str_filenamePath_10R_testSet_i in lst_str_filenamePath_10R_testSet:
                    print(lst_str_filenamePath_10R_testSet_i)
                print("lst_str_filenamePath_10R_validationSet: ")
                for lst_str_filenamePath_10R_validationSet_i in lst_str_filenamePath_10R_validationSet:
                    print(lst_str_filenamePath_10R_validationSet_i)

                print("lst_str_filenamePath_10R_trainingSet: ")
                for lst_str_filenamePath_10R_trainingSet_i in lst_str_filenamePath_10R_trainingSet:
                    print(lst_str_filenamePath_10R_trainingSet_i)
            
            
            
            
            #lst_test_masks=lst_str_filenamePath_10R_testSet_Masks
            #lst_test_images=lst_str_filenamePath_10R_testSet_Raws
            
            #lst_val_masks=lst_str_filenamePath_10R_validationSet_Masks
            #lst_val_images=lst_str_filenamePath_10R_validationSet_Raws
            
            lst_test_masks=sorted(lst_str_filenamePath_testSet_Masks)
            lst_test_images=sorted(lst_str_filenamePath_testSet_Raws)
            
            lst_val_masks=sorted(lst_str_filenamePath_validationSet_Masks)
            lst_val_images=sorted(lst_str_filenamePath_validationSet_Raws)
            
            
            lst_train_masks=sorted(lst_str_filenamePath_trainingSet_Masks)
            lst_train_images=sorted(lst_str_filenamePath_trainingSet_Raws)
  
        
        
        if False:
            int_ultimaImagen = int_NUM_TRAIN_IMAGES+ int_NUM_VAL_IMAGES + int_NUM_TEST_IMAGES
            #test_images0 = [sorted(glob(os.path.join(DATA_DIR, r"video_animal2_proyeccion_SeqDA/*")))[0]]
            #test_images0 = [sorted(glob(os.path.join(DATA_DIR, r"video_animal2_proyeccion_SeqDA/*")))[ultimaImagen]]
            test_images0 = [sorted(glob(os.path.join(str_DATA_DIR, str_RAW_Folder)))[int_ultimaImagen]]#20230123_1544
            #test_masks0 = [sorted(glob(os.path.join(DATA_DIR, r"EVLstack_SeqDA/*")))[0]]
            test_masks0 = [sorted(glob(os.path.join(str_DATA_DIR, str_MASK_Folder)))[int_ultimaImagen]]

        

        
        if ("Si estoy tirando cientos de casos, es mejor dejarlas en false porque printean mucho"!="Si estoy tirando cientos de casos, es mejor dejarlas en false porque printean mucho"):
            print("\x1b[34m\"A20220911_DeeplabVBS. Printear Imagenes originales que entrenaron, validaron y testearon modelo\"\x1b[0m")
            print("len(lst_train_masks): ", len(lst_train_masks))
            print("len(lst_val_masks): ", len(lst_val_masks))
            print("len(lst_test_masks): ", len(lst_test_masks))
            
            
            print("train_images: ")
            for lst_train_images_i in lst_train_images:
                print(lst_train_images_i)
            print("val_images: ")
            for lst_val_images_i in lst_val_images:
                print(lst_val_images_i)
            print("test_images: ")
            for lst_test_images_i in lst_test_images:
                print(lst_test_images_i)

        
        
        train_dataset = self.data_generator(lst_train_images, lst_train_masks)
        val_dataset = self.data_generator(lst_val_images, lst_val_masks)
        test_dataset = self.data_generator(lst_test_images, lst_test_masks)

        print("\x1b[34m\"Printear Datasets\"\x1b[0m")
        print("Train Dataset:", train_dataset)
        print("Val Dataset:", val_dataset)
        print("Test Dataset:", test_dataset)

        print("len(train_images)",len(lst_train_images))
        print("len(val_images)",len(lst_val_images))
        print("len(test_images)",len(lst_test_images))
        
        #print("main self.int_IMAGE_SIZE", self.int_IMAGE_SIZE)
        self.model = self.DeeplabV3Plus(image_size=self.int_IMAGE_SIZE, num_classes=self.int_NUM_CLASSES)
        #self.model.summary()
        
        tf.keras.utils.plot_model(self.model, show_shapes=True)
        
        
        #loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
        #loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False)#, ignore_class=255)#20230127_0850 , ignore_class=21
        if(str_DatasetNombre=="DatasetPascal"):
            loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False,ignore_class=21)#, ignore_class=255)#20230127_0850 ,ignore_class=21
        elif(str_DatasetNombre=="DatasetEmbrion"):
            loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False,ignore_class=None)#, ignore_class=255)#20230127_0850 ,ignore_class=21
        elif(str_DatasetNombre=="DatasetNeuron"):
            loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False,ignore_class=None)#, ignore_class=255)#20230127_0850 ,ignore_class=21
        elif(str_DatasetNombre=="DatasetEmbrionSinDA"):
            loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False,ignore_class=None)#, ignore_class=255)#20230127_0850 ,ignore_class=21
        elif(str_DatasetNombre=="DatasetTissue"):
            loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False,ignore_class=None)#, ignore_class=255)#20230127_0850 ,ignore_class=21
        
        
        else:
            raise Exception("Si str_data_dirIn no es PascalTrainVal ni tampoco COLAB_instance-level-human-parsing2/embryoDataset. Tengo que saber si es de pascal para poenrle ignore label en el if")
            
        
        callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
        
        self.model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.0001),
        loss=loss,
        metrics=["accuracy"],
        )

        
        if True: #Mostrar train 20230707. QUE NO SE TRASLAPEN LOS GRUPOS
            if False:
                print("Antes de entrar al entrenamiento")
                for lst_train_images_i, lst_train_masks_i in zip(lst_train_images, lst_train_masks):
                    print(r"lst_train_images_i: ", lst_train_images_i)
                    print(r"lst_train_masks_i: ", lst_train_masks_i)
                
                
            set_lst_train_images = set(lst_train_images)
            set_lst_val_images = set(lst_val_images)
            set_lst_test_images = set(lst_test_images)
     
           
            set_common_strings1 = set_lst_train_images.intersection(set_lst_val_images)
            set_common_strings2 = set_lst_train_images.intersection(set_lst_test_images)
            set_common_strings3 = set_lst_val_images.intersection(set_lst_test_images)

            print("set_common_strings1 Train Val: ", set_common_strings1)
            print("set_common_strings2 Train Test: ", set_common_strings2)
            print("set_common_strings3 Val Test: ", set_common_strings3)
            
            if((len(set_common_strings1)>0) or (len(set_common_strings2)>0) or (len(set_common_strings3)>0)):
                raise Exception("Se traslapan los conjuntos de training, validacion y testeo!")    



        if(self.str_entrenaroCargar=="Train"):#20230119_1734
            print("SE METIO A TRAIN")            
            
                  
            import time#202306100922
            flo_start_time = time.time()#202306100922            
            history = self.model.fit(train_dataset, validation_data=val_dataset, epochs=int_EPOCAS, callbacks=[callback])#original son 25 epochs
            flo_end_time = time.time()#202306100922
            
            plt.plot(history.history["loss"])
            plt.title("Training Loss")
            plt.ylabel("loss")
            plt.xlabel("epoch")
            plt.show()

            plt.plot(history.history["accuracy"])
            plt.title("Training Accuracy")
            plt.ylabel("accuracy")
            plt.xlabel("epoch")
            plt.show()

            plt.plot(history.history["val_loss"])
            plt.title("Validation Loss")
            plt.ylabel("val_loss")
            plt.xlabel("epoch")
            plt.show()

            plt.plot(history.history["val_accuracy"])
            plt.title("Validation Accuracy")
            plt.ylabel("val_accuracy")
            plt.xlabel("epoch")
            plt.show()
            
            int_num_epochs = len(history.epoch)#20230610
            print("Epochs used: ",int_num_epochs)#20230610
            
            if False:#20230613
                if(self.boo_guardarModeloIn==True):#20230119_1734
                    str_carpetaDir = str_data_dirIn.rsplit('/', 1)[-1]
                    from datetime import datetime as dt  #20230119_1734
                    str_fecha=dt.today().strftime('%Y%m%d_%H%M')#20230119_1734
                    str_prefijoModel=str_carpetaDir+"B"+str(int_batch_sizeIn)+"NC"+str(int_num_classesIn)+"Tra"+str(int_num_train_imagesIn)+"Val"+str(int_num_val_imagesIn)+"Tes"+str(int_num_test_imagesIn)+"Epo"+str(int_epocasIn)#20230119_1734
                    str_nombreSavedModel=r'saved_models/'+str_fecha+"_A20220911_class_DeepLabVBS_"+str_prefijoModel
                    #print(r"Guardando modelo en: ", r'saved_models/'+str_fecha+"class_DeepLab"+str_prefijoModel)
                    print(r"Guardando modelo en: ", str_nombreSavedModel)
                    #self.model.save(r'saved_models/'+str_fecha+"_A20220911_class_DeepLabVBS"+str_prefijoModel)#20230119_1734
                    self.model.save(str_nombreSavedModel)#20230119_1734
                       
        
        
        
        if(self.str_entrenaroCargar=="Load"): #20230119_1734    
            print("SE METIO A LOAD")
            print("Modelo a cargar: ", self.str_dirModelLoad)
            self.model=tf.keras.models.load_model(r"saved_models/"+self.str_dirModelLoad) #20230119_1734
            print("Modelo cargado")
            #return(self.model)#esperimental... deberia saltarse todo lo siguiente
            
        
        colormap = loadmat(
        r"human_colormap.mat"
        )["colormap"]
        colormap = colormap * 100
        colormap = colormap.astype(np.uint8)
        
        #self.plot_predictions(test_images[:4], colormap, model=self.model)
        #print("Plotear test images")
        
        if (True==False):
            print("\x1b[34m\"Plotear test images\"\x1b[0m")
            self.plot_predictions(lst_test_images, colormap, model=self.model)
        

        
        
        
        #CALCULAR MIOU INDIVIDUAL  
        if("CalcularMioUIndividual"=="CalcularMioUIndividual"):
            
            print("\x1b[34m\"ClassDeeplab: CalcularMioUIndividual\"\x1b[0m")
            
            
            
            self.test_masksIM = self.load_dataVBS(lst_test_masks, 
                                                  mask=True, 
                                                  to_tensor=False, 
                                                  img_Size1=self.int_IMAGE_SIZE)
            self.test_imagesIM = self.load_dataVBS(lst_test_images, 
                                                   mask=False, 
                                                   to_tensor=False, 
                                                   img_Size1=self.int_IMAGE_SIZE)
            
            print("Marcador1")
            self.lst_predsConArgmax=self.calc_predictionsVBSConArgmax(lst_test_images, 
                                                                      self.model, 
                                                                      img_Size2=self.int_IMAGE_SIZE)  
            
            
            print("Marcador2")
            self.lst_predsSinArgmax=self.calc_predictionsVBSSinArgmax(lst_test_images, 
                                                                      self.model, 
                                                                      img_Size2=self.int_IMAGE_SIZE)
            print("Marcador3")
            

            print("boo_activarCRFIn: ", boo_activarCRFIn)
            if (boo_activarCRFIn==True):#20230629
                print("ENTRO AL IF DEL CRF")
                print("ENTRO AL IF DEL CRF")
                print("ENTRO AL IF DEL CRF")
                raise Exception("Echate este error es que no quiero que tire crf en DEEPLAB, quiero estar seguro")  
                
                import import_ipynb
                from A20230112_CRF import class_CRF
                
                lst_predsConArgmaxCRF=[]
                for lst_predsSinArgmax_i, lst_test_images_i in zip(self.lst_predsSinArgmax, lst_test_images):
                    lst_predsSinArgmax_i=np.squeeze(lst_predsSinArgmax_i)
                    
                    nda_3DXX3_imgRGB = cv2.cvtColor(cv2.imread(lst_test_images_i), cv2.COLOR_BGR2RGB)##Asi esta en A20221210_pipeline5clase. LE estoy pasando al metodo CRF, con metodo CRF no he cargado las imagenes Raw con tensorflow, solo con cv2. Para deeplab si que frecuentemente se leen las imagenes con tensorflow y no con cv2
                    #print("ZZ1")
                    #print("nda_3DXX3_imgRGB.shape:", nda_3DXX3_imgRGB.shape)
                    #print("lst_predsSinArgmax_i.shape:", lst_predsSinArgmax_i.shape)
                    nda_3DXX3_imgRGB = cv2.resize(nda_3DXX3_imgRGB, (lst_predsSinArgmax_i.shape[0], lst_predsSinArgmax_i.shape[1]))#lst_predsSinArgmax8 tiene dimensiones anchox alto x numero de clases
                    #print("ZZ2")
                    #print("nda_3DXX3_imgRGB.shape:", nda_3DXX3_imgRGB.shape)
                    #print("lst_predsSinArgmax_i.shape:", lst_predsSinArgmax_i.shape)
                
                    
                    nda_3DXX2_ProbCRFmap = class_CRF().func_aplicar_CRF(nda_2DXX_sdXIn=-1,#Siempre debe ponerse 0 aqui porque es el metodo original...  El original no tiene estos parametros que hipoteticamente es lo mismo que dejarlos en 0
                                                                        nda_2DXX_sdYIn=-1,#Siempre debe ponerse 0 aqui porque es el metodo original... El original no tiene estos parametros que hipoteticamente es lo mismo que dejarlos en 0
                                                                        nda_3DXX2_probmapIn=lst_predsSinArgmax_i, 
                                                                        tpl_ParametroXYIn=(1,1),#20230607tpl_ParametroXYIn1, 
                                                                        #tpl_ParametroRGBIn=tpl_ParametroRGBIn1, 
                                                                        #tpl_ParametroXYIn=self.tpl_ParametroXYInFORZADO, #REDFLAG
                                                                        tpl_ParametroRGBIn=(1,1,1), 
                                                                        nda_3DXX3_imgRawIn=nda_3DXX3_imgRGB, 
                                                                        int_inferenceIn=7, 
                                                                        int_pairwisebilateralCompatIn=3,
                                                                        str_PairwiseoKernel="GaussianoSmoothnessKernelYBilateraloAppearanceKernel"
                                                                       )
                    nda_2DXX_ProbCRFmap_Argmax = np.argmax(nda_3DXX2_ProbCRFmap, axis=2)
                    lst_predsConArgmaxCRF.append(nda_2DXX_ProbCRFmap_Argmax)
                self.lst_predsConArgmax=lst_predsConArgmaxCRF.copy()

            
            
            
            
            #Aqui calcula el miou
            i=0
            self.lst_miouIND=[]
            for lst_preds_i in self.lst_predsConArgmax:  
                mi = tf.keras.metrics.MeanIoU(num_classes=int_num_classesIn)
                mi.update_state(self.test_masksIM[i], self.lst_predsConArgmax[i]) #True, pred
                aux_mi = (mi.result().numpy())

                self.lst_miouIND.append(aux_mi)
                i+=1

            dict_miouIND = {lst_test_images[i]: self.lst_miouIND[i] for i in range(len(lst_test_images))}


            # Printing resultant dictionary


            self.miouINDavg = sum(self.lst_miouIND) / len(self.lst_miouIND)    
            
        
            
            
            
            print("type(self.test_masksIM[0]: ", type(self.test_masksIM[0]))
            print("(self.test_masksIM[0]).shape: ", (self.test_masksIM[0]).shape)
            print("type(self.lst_predsConArgmax [0]: ", type(self.lst_predsConArgmax[0]))
            print("(self.lst_predsConArgmax[0]).shape ", (self.lst_predsConArgmax[0]).shape)

            print("\x1b[31m\"Class Deeplab: MIoU todo el data test individual \"\x1b[0m")
            #print("self.lst_miouIND (individual): ",self.lst_miouIND)
            print("len(self.lst_miouIND) (individual): ",len(self.lst_miouIND))
            #print("avg(iou) (individual): ", sum(self.lst_miou) / len(self.lst_miou))
            print("self.miouINDavg: ", self.miouINDavg)      
            #print("dict_miouIND: ", dict_miouIND)
            
            print("dict_miouIND: ")
            for item in dict_miouIND.items():
                print(item)

        
        
        if("CalcularMioUIndividualConLibreria"=="CalcularMioUIndividualConLibreria"):
            
            print("\x1b[34m\"ClassDeeplab: CalcularMioUIndividualConLibreria\"\x1b[0m")
            import import_ipynb
            from A20230105_MaskImageModel2mIoU import class_MaskImageModel2mIoU
            #i=0 20230613
            self.lst_miouINDconLib1=[]
            #self.lst_miouINDconLib2=[]
            #self.lst_miouINDconLib3=[]
            #self.lst_miouINDconLib4=[]#20230119_14:58
            self.lst_miouINDconLib5=[]
            #self.lst_miouINDconLib6=[] #20230121_1436
            self.lst_miouINDconLib7=[] #20230121_1436
            #self.lst_miouINDconLib8=[] #20230121_1436
            
            if(str_DatasetNombre=="DatasetPascal"):
                self.lst_miouINDconLib7_IL=[] #202300613 IGNORELABEL
            
            
            #listaFiles=os.listdir(r"./"+data_dirIn+r"/"+r"EVLstack_SeqDA"+r"/")
            #listaFiles=[x.split('.')[0] for x in listaFiles]
            #print("listaFiles: ",listaFiles)

            
            
            #dir=(sorted(glob(os.path.join(DATA_DIR, r"video_animal2_proyeccion_SeqDA/*"))))
            #print("dir")
            #for dir_i in dir:
            #    print(dir_i)
            if ("ordenados"==False):
                elordendelosarchivosordenados=("1OR, 90L, 90LF, 90R, 90RF, F, UD, UDF")
            
            #####OBTENER NOMBRE ARCHIVOS######
            #####OBTENER NOMBRE ARCHIVOS######
            #####OBTENER NOMBRE ARCHIVOS######
            print("lst_nombreFilesOut")
            
            #lst_nombreFiles=sorted(glob(r"./"+r"COLAB_instance-level-human-parsing2/embryoDataset"+r"/"+r"video_animal2_proyeccion_SeqDA/*"))
            lst_nombreFiles=sorted(glob(r"./"+r"COLAB_instance-level-human-parsing2/embryoDataset"+r"/"+r"video_animal2_proyeccion_SeqDA2/*"))#20230123_1544
            lst_nombreFiles=lst_test_images
            
            
            import re
            lst_nombreFilesOut=[]
            for lst_nombreFiles_i in lst_nombreFiles:
                aux=re.search(r"[^/]+$", lst_nombreFiles_i)[0]
                aux=re.match(r".*(?=\.)",aux)[0]
                lst_nombreFilesOut.append(aux)


            
                #print(aux)
            #print("lst_nombreFilesOut: ",lst_nombreFilesOut)
            #####OBTENER NOMBRE ARCHIVOS######
            #####OBTENER NOMBRE ARCHIVOS######
            #####OBTENER NOMBRE ARCHIVOS######
            
            
            i=0#20230613
            for lst_preds_i,lst_nombreFilesOut_i  in zip(self.lst_predsConArgmax,lst_nombreFilesOut): 
                
                print("lst_nombreFilesOut_i: ",lst_nombreFilesOut_i)
                
                

                
                
                
                
                
                
                
                
                
                
                #print("lst_preds_i: ",lst_preds_i)
                #mi = tf.keras.metrics.MeanIoU(num_classes=num_classesIn)
                #mi.update_state(test_masksIM[i], self.lst_predsConArgmax[i]) #True, pred
                #aux_mi = (mi.result().numpy())

                
                #print("objeto_class_MaImMo2mIoU().calcular_numpy_postPrediction_mIoU")
                #Esto deberia dar iouClass0=0/2=0; iouClass1=2/4=1/2; miou=(iou0+iou1)/2=1/4=0.25
                print("############")
                

                #print("objeto_class_MaImMo2mIoU().calcular_tf_postPrediction_mIoU")
                
                aux_mi1=class_MaskImageModel2mIoU().calcular_TFreadIm_model_TFmIoU_f1(str_directorioRaiz = str_data_dirIn, 
                                                                                      str_directorioMascara = str_mask_folderIn, #r"EVLstack_SeqDA", #20230610
                                                                                      #directorioImagen = r"video_animal2_proyeccion_SeqDA",
                                                                                      str_directorioImagen = str_raw_folderIn, #r"video_animal2_proyeccion_SeqDA2",#20230123_1544#20230610
                                                                                      #nombreArchivoImagen = r"UDF0085",                                        
                                                                                      str_nombreArchivoImagen = lst_nombreFilesOut_i,                                        
                                                                                      kerFunc_modelIn = self.model,
                                                                                      int_IMAGE_SIZEIn=self.int_IMAGE_SIZE,
                                                                                      int_num_classesIn=self.int_NUM_CLASSES)
                print("aux_mi1", aux_mi1)
                print("###############")
                
                if False:
                    aux_mi2=class_MaskImageModel2mIoU().calcular_TFreadIm_model_NPmIoU2Clases_f2(str_directorioRaiz = str_data_dirIn,#20230121_0345 
                                                                                                 str_directorioMascara = r"EVLstack_SeqDA", #20230121_0345
                                                                                                 #directorioImagen = r"video_animal2_proyeccion_SeqDA",#20230121_0345
                                                                                                 str_directorioImagen = r"video_animal2_proyeccion_SeqDA2",#20230123_1544
                                                                                                 #nombreArchivoImagen = r"UDF0085",                                        
                                                                                                 str_nombreArchivoImagen = lst_nombreFilesOut_i,#20230121_0345
                                                                                                 kerFunc_modelIn = self.model)#20230121_0345
                    print("aux_mi2", aux_mi2)

                if False:
                    print("###############")
                    aux_mi3=class_MaskImageModel2mIoU().calcular_CV2readIm_model_TFmIoU_f3(str_directorio_imgMask=str_DATA_DIR+r"/"+r"EVLstack_SeqDA"+r"/"+lst_nombreFilesOut_i+r".png", #20230119_14:58
                                                                                           #directorioimgTest=DATA_DIR+r"/"+r"video_animal2_proyeccion_SeqDA"+r"/"+lst_nombreFilesOut_i+r".jpg",#20230119_14:58
                                                                                           str_directorioimgTest=str_DATA_DIR+r"/"+r"video_animal2_proyeccion_SeqDA2"+r"/"+lst_nombreFilesOut_i+r".png",#20230121_0345
                                                                                           kerFunc_modelIn=self.model,
                                                                                           int_num_classesIn=self.int_NUM_CLASSES)#20230119_14:58
                    print("aux_mi3", aux_mi3)
                
                    print("###############")
                    aux_mi4=class_MaskImageModel2mIoU().calcular_CV2readIm_model_NPmIoU2Clases_f4(str_directorio_imgMask=str_DATA_DIR+r"/"+r"EVLstack_SeqDA"+r"/"+lst_nombreFilesOut_i+r".png", #20230119_14:58
                                                                                                  #directorioimgTest=str_DATA_DIR+r"/"+r"video_animal2_proyeccion_SeqDA"+r"/"+lst_nombreFilesOut_i+r".jpg",#20230119_14:58
                                                                                                  str_directorioimgTest=str_DATA_DIR+r"/"+r"video_animal2_proyeccion_SeqDA2"+r"/"+lst_nombreFilesOut_i+r".png",#20230121_0345
                                                                                                  kerFunc_modelIn=self.model)#20230119_14:58
                    print("aux_mi4", aux_mi4)
                
                print("###############")
                aux_mi5=class_MaskImageModel2mIoU().calcular_model_TFmIoU_f5(et_3DXX1_imgMaskIn=self.test_masksIM[i], 
                                                                             et_3DXX3_imgTestIn=self.test_imagesIM[i], 
                                                                             kerFunc_modelIn=self.model, 
                                                                             int_num_classesIn=self.int_NUM_CLASSES)
                print("aux_mi5", aux_mi5)
                
                if False:
                    print("###############")
                    aux_mi6=class_MaskImageModel2mIoU().calcular_model_NPmIoU2Clases_f6(et_3DXX1_imgMaskIn=self.test_masksIM[i], 
                                                                                        et_3DXX3_imgTestIn=self.test_imagesIM[i], 
                                                                                        kerFunc_modelIn=self.model)
                    print("aux_mi6", aux_mi6)
                
                print("###############")
                aux_mi7=class_MaskImageModel2mIoU().calcular_postPrediction_TFmIoU_f7(et_3DXX1_imgMaskIn=self.test_masksIM[i], 
                                                                                      nda_2DXX_imgTestPredictedIn=self.lst_predsConArgmax[i], 
                                                                                      int_num_classesIn=self.int_NUM_CLASSES)            
                print("aux_mi7", aux_mi7)
                
                if(str_DatasetNombre=="DatasetPascal"):
                    aux_mi7_IL=class_MaskImageModel2mIoU().calcular_postPrediction_TFmIoU_f7(et_3DXX1_imgMaskIn=self.test_masksIM[i], 
                                                                                          nda_2DXX_imgTestPredictedIn=self.lst_predsConArgmax[i], 
                                                                                          int_num_classesIn=self.int_NUM_CLASSES,
                                                                                          nt_int_ignoreLabelIn=21)            
                    print("aux_mi7_IL", aux_mi7_IL)
                    
                    
                    
                if False:
                    print("###############")
                    aux_mi8=class_MaskImageModel2mIoU().calcular_postPrediction_NPmIoU2Clases_f8(et_3DXX1_imgMaskIn=self.test_masksIM[i], 
                                                                                                 nda_2DXX_imgTestPredictedIn=self.lst_predsConArgmax[i])
                    print("aux_mi8", aux_mi8)
                print("###############")
                print("###############")
                print("###############")
                 
                
                self.lst_miouINDconLib1.append(aux_mi1)
                #self.lst_miouINDconLib2.append(aux_mi2)
                #self.lst_miouINDconLib3.append(aux_mi3)
                #self.lst_miouINDconLib4.append(aux_mi4)#20230119_14:58
                self.lst_miouINDconLib5.append(aux_mi5)#20230121_0345
                #self.lst_miouINDconLib6.append(aux_mi6)#20230121_1436
                self.lst_miouINDconLib7.append(aux_mi7)
                #self.lst_miouINDconLib8.append(aux_mi8)
                if(str_DatasetNombre=="DatasetPascal"):
                    self.lst_miouINDconLib7_IL.append(aux_mi7_IL)
                
                i+=1

            dict_miouINDconLib1 = {lst_test_images[i]: self.lst_miouINDconLib1[i] for i in range(len(lst_test_images))}
            #dict_miouINDconLib2 = {lst_test_images[i]: self.lst_miouINDconLib2[i] for i in range(len(lst_test_images))}
            #dict_miouINDconLib3 = {lst_test_images[i]: self.lst_miouINDconLib3[i] for i in range(len(lst_test_images))}
            #dict_miouINDconLib4 = {lst_test_images[i]: self.lst_miouINDconLib4[i] for i in range(len(lst_test_images))}#20230119_14:58
            dict_miouINDconLib5 = {lst_test_images[i]: self.lst_miouINDconLib5[i] for i in range(len(lst_test_images))}#20230121_0345
            #dict_miouINDconLib6 = {lst_test_images[i]: self.lst_miouINDconLib6[i] for i in range(len(lst_test_images))}#20230121_1436
            dict_miouINDconLib7 = {lst_test_images[i]: self.lst_miouINDconLib7[i] for i in range(len(lst_test_images))}#20230121_1436
            #dict_miouINDconLib8 = {lst_test_images[i]: self.lst_miouINDconLib8[i] for i in range(len(lst_test_images))}#20230121_1436
            if(str_DatasetNombre=="DatasetPascal"):
                dict_miouINDconLib7_IL = {lst_test_images[i]: self.lst_miouINDconLib7_IL[i] for i in range(len(lst_test_images))}#20230121_1436

            print("este es como el promedio")
            self.miouINDconLib1 = sum(self.lst_miouINDconLib1) / len(self.lst_miouINDconLib1)  
            #self.miouINDconLib2 = sum(self.lst_miouINDconLib2) / len(self.lst_miouINDconLib2)  
            #self.miouINDconLib3 = sum(self.lst_miouINDconLib3) / len(self.lst_miouINDconLib3) 
            #self.miouINDconLib4 = sum(self.lst_miouINDconLib4) / len(self.lst_miouINDconLib4) #20230119_14:58
            self.miouINDconLib5 = sum(self.lst_miouINDconLib5) / len(self.lst_miouINDconLib5) #20230121_0345
            #self.miouINDconLib6 = sum(self.lst_miouINDconLib6) / len(self.lst_miouINDconLib6) #20230121_1436
            self.miouINDconLib7 = sum(self.lst_miouINDconLib7) / len(self.lst_miouINDconLib7) #20230121_1436
            #self.miouINDconLib8 = sum(self.lst_miouINDconLib8) / len(self.lst_miouINDconLib8) #20230121_1436
            if(str_DatasetNombre=="DatasetPascal"):
                self.miouINDconLib7_IL = sum(self.lst_miouINDconLib7_IL) / len(self.lst_miouINDconLib7_IL) #20230121_1436
                
            # Printing resultant dictionary
            
            print("Experimental miou promedio self.miouINDconLib1: ", self.miouINDconLib1)
            print("Experimental miou promedio self.miouINDconLib5: ", self.miouINDconLib5)
            print("Experimental miou promedio self.miouINDconLib7: ", self.miouINDconLib7)
            if(str_DatasetNombre=="DatasetPascal"):
                print("Experimental miou promedio self.miouINDconLib7_IL: ", self.miouINDconLib7_IL)
            
            if False:
                #print("dict_miouINDconLib1: ")
                print("\x1b[34m\"Class Deeplab: calcular_numpy_postPrediction_mIoU\"\x1b[0m")
                for item in dict_miouINDconLib1.items():
                    print(item*100)

                #print("dict_miouINDconLib2: ")
                print("\x1b[34m\"Class Deeplab: calcular_tf_postPrediction_mIoU\"\x1b[0m")
                for item in dict_miouINDconLib2.items():
                    print(item*100)

                print("\x1b[34m\"Class Deeplab: calcular_tf_readIm_tf_model_mIoUl\"\x1b[0m")
                for item in dict_miouINDconLib3.items():
                    print(item*100)

                print("\x1b[34m\"Class Deeplab: calcular_opencv_readIm_tf_model_mIoU\"\x1b[0m")#20230119_14:58
                for item in dict_miouINDconLib4.items():#20230119_14:58
                    print(item*100)#20230119_14:58

            
        
        #MIOU SOLO DE LA PRIMERA IMAGEN
        #MIOU SOLO DE LA PRIMERA IMAGEN
        #MIOU SOLO DE LA PRIMERA IMAGEN
        
        

        if("PrimeraImagenMioU"=="PrimeraImagenMioU"):
        #if(False):
            lst_test_masks0=[lst_test_masks[0]]
            lst_test_images0=[lst_test_images[0]]
            self.test_masksIM0 = self.load_dataVBS(lst_test_masks0, mask=True, to_tensor=False, img_Size1=self.int_IMAGE_SIZE)
            self.lst_predsConArgmax0=self.calc_predictionsVBSConArgmax(lst_test_images0, self.model, img_Size2=self.int_IMAGE_SIZE) 
            self.lst_predsSinArgmax0=self.calc_predictionsVBSSinArgmax(lst_test_images0, self.model, img_Size2=self.int_IMAGE_SIZE)#es una lista por eso la modifico abajo     
            self.lst_predsSinArgmax0=self.lst_predsSinArgmax0[0]
        
            m0 = tf.keras.metrics.MeanIoU(num_classes=int_num_classesIn)
            #i=0
            #for lst_preds_i in self.lst_preds0: 
            m0.update_state(self.test_masksIM0, self.lst_predsConArgmax0)  
                #m0.update_state(test_masksIM[i], self.lst_preds[i]) 
            #    i+=1
            self.miou0=m0.result().numpy()*100
            print("\x1b[34m\"Class Deeplab: MIoU solo de la primera imagen\"\x1b[0m")
            print("MIoU solo de la primera imagen usando self.test_masksIM0, self. lst_predsConArgmax0: ",lst_test_masks0)
            print("MIoU solo de la primera imagen usando self.test_masksIM0, self. lst_predsConArgmax0: ",self.miou0)
            
            
        if(self.str_entrenaroCargar=="Train"):
            if(self.boo_guardarModeloIn==True):#20230119_1734
                flo_rounded_miou = round(self.miouINDconLib7, 2)
                if(str_DatasetNombre=="DatasetPascal"):
                    flo_rounded_miou = round(self.miouINDconLib7_IL, 2)
                    
                str_rounded_miou = str(flo_rounded_miou).replace(".", "_")

                flo_duration_seconds = flo_end_time - flo_start_time
                flo_duration_hours = flo_duration_seconds / 3600  # Convert seconds to hours
                flo_round_duration_hours = round(flo_duration_hours, 1)
                str_round_duration_hours = str(flo_round_duration_hours).replace(".", "_")

                #str_carpetaDir = str_data_dirIn.rsplit('/', 1)[-1]

                #str_prefijoModel=str_carpetaDir+"Miou"+str(self.miouINDconLib7)+"B"+str(int_batch_sizeIn)+"NC"+str(int_num_classesIn)+"Tra"+str(int_num_train_imagesIn)+"Val"+str(int_num_val_imagesIn)+"Tes"+str(int_num_test_imagesIn)+"Epo"+str(int_epocasIn)#20230119_1734
                str_prefijoModel=(str_DatasetNombre+
                                 "Ba"+str(int_batch_sizeIn)+
                                 "NC"+str(int_num_classesIn)+
                                 #"Se"+str(int_seed)+
                                 "Tr"+str(int_num_train_imagesIn)+
                                 "Va"+str(int_num_val_imagesIn)+
                                 "Te"+str(int_num_test_imagesIn)+
                                 "Ep"+str(int_num_epochs)+#20230613
                                 "D"+str_round_duration_hours+
                                 "Mi"+str(str_rounded_miou)
                                 )


                str_nombreSavedModel=r'saved_models/'+str_fechaInicio+"_A20220911_class_DeepLabVBS_"+str_prefijoModel#
                #print(r"Guardando modelo en: ", r'saved_models/'+str_fecha+"class_DeepLab"+str_prefijoModel)
                print(r"Guardando modelo en: ", str_nombreSavedModel)
                print("Training duration:", str_round_duration_hours)
                #self.model.save(r'saved_models/'+str_fecha+"_A20220911_class_DeepLabVBS"+str_prefijoModel)#20230119_1734
                self.model.save(str_nombreSavedModel)#20230119_1734

                #with open(str_nombreSavedModel+r"/"+"Description.txt", 'w') as file:
                with open(str_nombreSavedModel+r"/"+r"DescriptionVBS.txt", 'w') as file:#20230613
                    file.write(r"str_nombreSavedModel: "+str_nombreSavedModel)#20230613
                    file.write("\r\n")#20230613
                    file.write(r"self.int_IMAGE_SIZE: "+str(self.int_IMAGE_SIZE))#20230613
                    file.write("\r\n")#20230613
                    file.write(r"int_seed: "+str(int_seed))#20230613
                
                with open(str_nombreSavedModel+r"/"+r"DatasetsFiles.txt", 'w') as file:#20230613
                    file.write(r"Training files: ")
                    file.write("\r\n")
                    for lst_train_images_i in lst_train_images:
                        file.write(lst_train_images_i)
                        file.write("\r\n")
                    file.write(r"Validation files: ")
                    file.write("\r\n")
                    for lst_val_images_i in lst_val_images:
                        file.write(lst_val_images_i)
                        file.write("\r\n")
                    file.write(r"Testing files: ")
                    file.write("\r\n")
                    for lst_test_images_i in lst_test_images:
                        file.write(lst_test_images_i)
                        file.write("\r\n")
        
        boo_Plotear=False#20230613
        if(boo_Plotear==True):#20230613
            
            if(str_DatasetNombre=="DatasetPascal"):
                i=0
                for lst_preds_i,lst_nombreFilesOut_i,(key_i, value_i)  in zip(self.lst_predsConArgmax,lst_nombreFilesOut, dict_miouINDconLib7_IL.items()):#20230614 dict_miouINDconLib7_IL este tiene ignore label
                    print(lst_nombreFilesOut_i)
                    self.func_plotearPascal(self.lst_predsConArgmax[i],
                                                 str_imgIn=key_i,
                                                 flo_miouIn=value_i
                                                )
                    i=i+1

                
                
            if((str_DatasetNombre=="DatasetEmbrion") or (str_DatasetNombre=="DatasetNeuron") or (str_DatasetNombre=="DatasetEmbrionSinDA")):#20230703
                i=0
                for lst_preds_i,lst_nombreFilesOut_i,(key_i, value_i)  in zip(self.lst_predsConArgmax,lst_nombreFilesOut, dict_miouINDconLib7.items()):#20230614 Este dict_miouINDconLib7 no tiene ignore label
                    print("Entro al plot de embrion")
                    self.func_plotearEmbrion(self.lst_predsConArgmax[i],
                                             str_imgIn=key_i,
                                             flo_miouIn=value_i
                                            )
                    i=i+1
            

        return(self.model)#20230610       

In [12]:
if False:    
    import numpy as np
    np.random.seed(11)

    my_list = [1, 2, 3, 4, 5]
    sample = np.random.choice(my_list, size=3, replace=False)
    print(sample)  # Output: a random sample of 3 elements from my_list
    sample = np.random.choice(my_list, size=3, replace=False)
    print(sample)


In [13]:
if True:    
    import numpy as np
    np.random.seed(11)

    my_list = list(set(["1", "2", "3", "4", "5"]))
    sample = np.random.choice(my_list, size=3, replace=False)
    print(sample)  # Output: a random sample of 3 elements from my_list
    sample = np.random.choice(my_list, size=3, replace=False)
    print(sample)


['3' '4' '5']
['3' '5' '4']


In [14]:
#my_dict = {"key1": "value1", 
#           "key2": "value2", 
#           "key3": "value3"}
#
#my_dict.items()

In [15]:
#my_dict = {"key1": "value1", "key2": "value2", "key3": "value3"}



In [16]:
#first_item = next(iter(my_dict.items()))

#print(first_item)

In [17]:
class class_DeepLab(class_DeepLab):

    def convolution_block(
        self,
        block_input,
        num_filters=256,
        kernel_size=3,
        dilation_rate=1,
        padding="same",
        use_bias=False,
    ):
        x = layers.Conv2D(
            num_filters,
            kernel_size=kernel_size,
            dilation_rate=dilation_rate,
            padding="same",
            use_bias=use_bias,
            kernel_initializer=keras.initializers.HeNormal(),
        )(block_input)
        x = layers.BatchNormalization()(x)
        return tf.nn.relu(x)


    def DilatedSpatialPyramidPooling(self, dspp_input):
        dims = dspp_input.shape
        x = layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
        x = self.convolution_block(x, kernel_size=1, use_bias=True)
        out_pool = layers.UpSampling2D(
            size=(dims[-3] // x.shape[1], dims[-2] // x.shape[2]), interpolation="bilinear",
        )(x)

        out_1 = self.convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
        out_6 = self.convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
        out_12 = self.convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
        out_18 = self.convolution_block(dspp_input, kernel_size=3, dilation_rate=18)

        x = layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
        output = self.convolution_block(x, kernel_size=1)
        return output


In [18]:
class class_DeepLab(class_DeepLab):
    
    def DeeplabV3Plus(self, 
                      image_size, 
                      num_classes
                     ):
        
        #print("image_size: ", image_size)
        model_input = keras.Input(shape=(image_size, image_size, 3))
        resnet50 = keras.applications.ResNet50(
            weights="imagenet", include_top=False, input_tensor=model_input
        )
        x = resnet50.get_layer("conv4_block6_2_relu").output
        x = self.DilatedSpatialPyramidPooling(x)

        input_a = layers.UpSampling2D(
            size=(image_size // 4 // x.shape[1], image_size // 4 // x.shape[2]),
            interpolation="bilinear",
        )(x)
        input_b = resnet50.get_layer("conv2_block3_2_relu").output
        input_b = self.convolution_block(input_b, num_filters=48, kernel_size=1)

        x = layers.Concatenate(axis=-1)([input_a, input_b])
        x = self.convolution_block(x)
        x = self.convolution_block(x)
        x = layers.UpSampling2D(
            size=(image_size // x.shape[1], image_size // x.shape[2]),
            interpolation="bilinear",
        )(x)
        model_output = layers.Conv2D(num_classes, kernel_size=(1, 1), padding="same", activation="softmax")(x)
        return keras.Model(inputs=model_input, outputs=model_output)


    

In [19]:
class class_DeepLab(class_DeepLab):
    
    def read_image(self,
                   image_path, 
                   img_Size,
                   mask=False,
                   ):
        """Lee imagenes individuales, no listas"""
        #self.func_datosVariable(image_path)
        #print("read_image self.int_IMAGE_SIZE", self.int_IMAGE_SIZE)
    
        image = tf.io.read_file(image_path)
        if mask:
            image = tf.image.decode_png(image, channels=1)
            image.set_shape([None, None, 1])
            #image = tf.image.resize(images=image, size=[self.int_IMAGE_SIZE, self.int_IMAGE_SIZE])
            image = tf.image.resize(images=image, size=[img_Size, img_Size], method="nearest")#20230610
        else:
            image = tf.image.decode_png(image, channels=3)
            image.set_shape([None, None, 3])
            image = tf.image.resize(images=image, size=[img_Size, img_Size])
            #image = tf.image.resize(images=image, size=[self.int_IMAGE_SIZE, self.int_IMAGE_SIZE], method="nearest")#20230610
            image = image / 127.5 - 1
        return image



    def load_data(self, 
                  image_list, 
                  mask_list,
                  #img_Size7
                 ):
        
        img_Size7=self.int_IMAGE_SIZE#YellowFlag
        image = self.read_image(image_list,
                               img_Size7)
        mask = self.read_image(mask_list,
                               img_Size7,
                               mask=True)
        return image, mask


    def data_generator(self, 
                       image_list, 
                       mask_list):
        
        dataset = tf.data.Dataset.from_tensor_slices((image_list, mask_list))
        dataset = dataset.map(self.load_data, num_parallel_calls=tf.data.AUTOTUNE)
        dataset = dataset.batch(self.int_BATCH_SIZE, drop_remainder=True)
        return dataset


In [20]:
class class_DeepLab(class_DeepLab):

    def inferConArgmax(self, 
                       model, 
                       image_tensor):
        """
        Inputs
        model: 'keras.engine.functional.Functional'
        image_tensor: tensorflow.python.framework.ops.EagerTensor
        Outputs:
        type(predictions):  <class 'numpy.ndarray'>
        
        Lee imagenes individuales, no listas
        """
        
        #print("inferConArgmax")
        #print("inferConArgmax type(model): ",type(model))
        #print("hambre0")
        predictions = model.predict(np.expand_dims((image_tensor), 
                                                   axis=0))#Aqui muere especificamente
        #print("hambre1")
        predictions = np.squeeze(predictions)
        #print("hambre2")
        predictions = np.argmax(predictions, axis=2)
        #print("hambre3")
        #print("inferConArgmax type(predictions): ",type(predictions))
        return predictions

    def calc_predictionsVBSConArgmax(self, 
                                     images_list, 
                                     model, 
                                     img_Size2
                                    ):
        """
        Inputs
        images_list: class 'list'
        model: class 'keras.engine.functional.Functional'
        Outputs:
        output: class 'list'
        
        Lee listas de imagenes individuales
        """
        
        #print("calc_predictionsVBSConArgmax")
        #print("calc_predictionsVBSConArgmax type(images_list): ",type(images_list))
        #print("calc_predictionsVBSConArgmax type(model): ",type(model))
        output=[]
        for image_file in images_list:
            #print("chao")
            #print("image_file: ",image_file)
            #print("img_Size2: ",img_Size2)
            image_tensor = self.read_image(image_file, 
                                           img_Size2)
            #print("chao2")
            
            if False:#20230610
                str_dirModelLoadIn9=r"20230709_0917_A20220911_class_DeepLabVBS_DatasetEmbrionBa3NC2Tr105Va10Te30Ep29D0_6Mi75_8"
                str_imgRawPathIn9=r"./COLAB_instance-level-human-parsing2/embryoDataset/video_animal2_proyeccion_SeqDA2/1OR0000.png"
                image = tf.io.read_file(str_imgRawPathIn9)
                image = tf.image.decode_png(image, channels=3)
                image.set_shape([None, None, 3])
                #image = tf.image.resize(images=image, size=[1024,1024])#20230610

                nda_3DXX3_uint8_imgRGB = image.numpy()
                Func_model=tf.keras.models.load_model(r"saved_models/"+str_dirModelLoadIn9) #20230119_1734
                print("basket")
                Func_model.predict(np.expand_dims((nda_3DXX3_uint8_imgRGB / 127.5 - 1), 
                                           axis=0))#Loagregue 20290929
                print("futbol")
                
            #print("almuerzo")
            prediction_mask = self.inferConArgmax(image_tensor=image_tensor, #AQUI SE CAE 20230929
                                                  model=model)
            #print("chao3")
            output.append(prediction_mask)
        #print("calc_predictionsVBSConArgmax type(output): ",type(output))
        return(output)
          #prediction_colormap = decode_segmentation_masks(prediction_mask, colormap, 20)
          #overlay = get_overlay(image_tensor, prediction_colormap)
          #plot_samples_matplotlib(
          #    [image_tensor, overlay, prediction_colormap], figsize=(18, 14)
          #)
        
    def calc_predictionsVBSSinArgmax(self, 
                                     images_list, 
                                     model,
                                    img_Size2
                                    ):
        """
        Inputs
        images_list: class 'list'
        model: class 'keras.engine.functional.Functional'
        Outputs:
        output: class 'list'. Lista con cada elemento de dimensiones (batchxaltoxanchox2)
        
        Lee listas de imagenes individuales
        """
        
        #print("calc_predictionsVBSSinArgmax")
        def inferSinArgmax(model, image_tensor):
            """
            Inputs
            model: class 'keras.engine.functional.Functional'
            image_tensor: class 'tensorflow.python.framework.ops.EagerTensor'
            Outputs:
            predictions: class 'numpy.ndarray'
            
            Lee imagenes individuales
            """
            
            
            #print("inferSinArgmax inferSinArgmax")
            #print("inferSinArgmax type(image_tensor): ",type(image_tensor))
            #print("inferSinArgmax type(model): ",type(model))
            predictions = model.predict(np.expand_dims((image_tensor), axis=0))
            #print("inferSinArgmax type(predictions): ",type(predictions))
            return predictions
        
        #print("calc_predictionsVBSSinArgmax type(images_list): ",type(images_list))
        #print("calc_predictionsVBSSinArgmax type(model): ",type(model))
        output=[]
        for image_file in images_list:
            print("hola")
            image_tensor = self.read_image(image_file,
                                          img_Size2)#va a tirar error porque no tiene imgsize de parametro
            print("hola2")
            prediction_mask = inferSinArgmax(image_tensor=image_tensor, model=model)
            output.append(prediction_mask)
        #print("calc_predictionsVBSSinArgmax type(output): ",type(output))
        return(output)

    
    
    
    
    

In [21]:
class class_DeepLab(class_DeepLab):
    
    def plot_predictions(self, images_list, colormap, model):
        
        def decode_segmentation_masks(mask, colormap, n_classes):
            r = np.zeros_like(mask).astype(np.uint8)
            g = np.zeros_like(mask).astype(np.uint8)
            b = np.zeros_like(mask).astype(np.uint8)
            for l in range(0, n_classes):
                idx = mask == l
                r[idx] = colormap[l, 0]
                g[idx] = colormap[l, 1]
                b[idx] = colormap[l, 2]
            rgb = np.stack([r, g, b], axis=2)
            return rgb
        
        def get_overlay(image, colored_mask):
            image = tf.keras.preprocessing.image.array_to_img(image)
            image = np.array(image).astype(np.uint8)
            overlay = cv2.addWeighted(image, 0.35, colored_mask, 0.65, 0)
            return overlay             
    
        
        def plot_samples_matplotlib(display_list, figsize=(5, 3)):
            _, axes = plt.subplots(nrows=1, ncols=len(display_list), figsize=figsize)
            for i in range(len(display_list)):
                if display_list[i].shape[-1] == 3:
                    axes[i].imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
                else:
                    axes[i].imshow(display_list[i])
            plt.show()

        
        for image_file in images_list:
            image_tensor = self.read_image(image_file)#va a tirar error porque no tiene imgsize de parametro
            prediction_mask = self.inferConArgmax(image_tensor=image_tensor, model=model)
            prediction_colormap = decode_segmentation_masks(prediction_mask, colormap, 20)
            overlay = get_overlay(image_tensor, prediction_colormap)
            plot_samples_matplotlib(
              [image_tensor, overlay, prediction_colormap], figsize=(18, 14)
            )

In [22]:
class class_DeepLab(class_DeepLab):

    def load_dataVBS(self, 
                     image_list, 
                     mask, 
                     to_tensor,
                     img_Size1):
        """
        Inputs
        image_list: 'class 'list''
        mask: class 'bool'
        to_tensor: ?
        
        Outputs:
        output: 'class 'list''
        
        Outputs: Lista de read images
        """
        
        #print("load_dataVBS")
        #print("load_dataVBS type(image_list): ",type(image_list))
        #print("load_dataVBS type(mask): ",type(mask))
        output=[]
        for image_list_i in image_list:
            image = self.read_image(image_list_i, 
                                    img_Size1,
                                    mask)
            output.append(image)

        #if(to_tensor==True):
        #    output=tf.convert_to_tensor(output)
        #print("load_dataVBS type(output): ",type(output))
        return output
    

In [23]:
if False:
    class class_DeepLab(class_DeepLab):

        def func_datosVariable(self, variableIn):
            #strvariableIn=str(variableIn)
            #strvariableIn = f'{variableIn=}'.split('=')[0]
            strvariableIn="variableIn"

            print("type("+strvariableIn+r"): ",type(variableIn))
            print(strvariableIn+r".shape: ",variableIn.shape)
            print(r"np.min("+strvariableIn+r"): ",np.min(variableIn))
            print(r"np.max("+strvariableIn+r"): ",np.max(variableIn))
            print(r"np.unique("+strvariableIn+r"): ",np.unique(variableIn))
            print(r"len(np.unique("+strvariableIn+r")): ",len(np.unique(variableIn)))


In [24]:
class class_DeepLab(class_DeepLab):        

    def func_plotearPascal(self,
                           nda_2DXX_ImgIn,
                           str_imgIn,#para el titulo
                           flo_miouIn#para el titulo
                           ):
        import matplotlib.pyplot as plt
        import numpy as np

        # Define the color palette for PASCAL VOC 21 classes
        colors = [
            [0, 0, 0],        # Background (black)
            [128, 0, 0],      # Aeroplane (maroon)
            [0, 128, 0],      # Bicycle (green)
            [128, 128, 0],    # Bird (olive)
            [0, 0, 128],      # Boat (navy)
            [128, 0, 128],    # Bottle (purple)
            [0, 128, 128],    # Bus (teal)
            [128, 128, 128],  # Car (gray)
            [64, 0, 0],       # Cat (dark maroon)
            [192, 0, 0],      # Chair (dark red)
            [64, 128, 0],     # Cow (dark green)
            [192, 128, 0],    # Dining Table (dark olive)
            [64, 0, 128],     # Dog (dark purple)
            [192, 0, 128],    # Horse (dark magenta)
            [64, 128, 128],   # Motorbike (dark cyan)
            [192, 128, 128],  # Person (light gray)
            [0, 64, 0],       # Potted Plant (dark green)
            [128, 64, 0],     # Sheep (brown)
            [0, 192, 0],      # Sofa (green)
            [128, 192, 0],    # Train (olive)
            [0, 64, 128],     # TV/Monitor (dark blue)
            [255, 255, 255],  # IgnoreLabel
        ]

        # Define the class names for PASCAL VOC 21 classes
        class_names = [
            "Background",
            "Aeroplane",
            "Bicycle",
            "Bird",
            "Boat",
            "Bottle",
            "Bus",
            "Car",
            "Cat",
            "Chair",
            "Cow",
            "Dining Table",
            "Dog",
            "Horse",
            "Motorbike",
            "Person",
            "Potted Plant",
            "Sheep",
            "Sofa",
            "Train",
            r"TV/Monitor",
            "IgnoreLabel"
        ]

        # Convert color values from 0-255 to the range of 0-1
        colors = np.array(colors) / 255.0

        # Generate example data with square image dimensions
        #data = np.random.randint(0, 21, size=(100, 100))
        #data = np.repeat(np.arange(21).reshape(-1, 1), 21, axis=1)

        # Get unique values in data and their corresponding indices
        unique_values, indices = np.unique(nda_2DXX_ImgIn, return_inverse=True)

        # Map the indices to colors
        unique_colors = colors[unique_values]
        
        
        plt.figure(figsize=(10, 10))
        # Plot using the PASCAL VOC color palette
        #plt.imshow(nda_2DXX_ImgIn, cmap=plt.cm.colors.ListedColormap(colors))
        plt.imshow(indices.reshape(nda_2DXX_ImgIn.shape), cmap=plt.cm.colors.ListedColormap(unique_colors))#20230615 
        #plt.colorbar(ticks=np.arange(21))



        # Define legend data
        legend_data = [
            [i, colors[i], class_names[i]]
            for i in range(len(colors))
        ]

        # Create legend handles and labels
        handles = [
            plt.Rectangle((0, 0), 1, 1, color=c)
            for k, c, n in legend_data
        ]
        labels = [n for k, c, n in legend_data]
        
        ##Para mostrarlo en la legenda
        lst_uniques=np.unique(nda_2DXX_ImgIn).tolist()
        lst_str_uniquesNames=[]
        for lst_uniques_i in lst_uniques:
            print("lst_uniques_i", lst_uniques_i)
            print("class_names[lst_uniques_i]", class_names[lst_uniques_i])
            lst_str_uniquesNames.append(class_names[lst_uniques_i])
        ##Para mostrarlo en la legenda
        
        #print("miou: ",miouIn)
        # Display legend
        plt.legend(handles=handles, labels=labels, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
        plt.title(str(lst_uniques)+str(lst_str_uniquesNames)+str_imgIn+str(flo_miouIn))
        plt.show()
        
        print("Ojo esta malo en general este, muestra color blanco en otras clases")#YellowFLAG


In [25]:
class class_DeepLab(class_DeepLab):        

    def func_plotearEmbrion(self,
                           nda_2DXX_ImgIn,
                           str_imgIn,
                           flo_miouIn
                           ):
        import matplotlib.pyplot as plt
        import numpy as np

        # Define the color palette for PASCAL VOC 21 classes
        colors = [
            [0, 0, 0],        # Background (black)
            [255, 255, 255],  # Embrion
        ]

        # Define the class names for PASCAL VOC 21 classes
        class_names = [
            "Background",
            "Embryo",
        ]

        # Convert color values from 0-255 to the range of 0-1
        colors = np.array(colors) / 255.0
        
        # Get unique values in data and their corresponding indices
        unique_values, indices = np.unique(nda_2DXX_ImgIn, return_inverse=True)

        # Map the indices to colors
        unique_colors = colors[unique_values]
        

        # Generate example data with square image dimensions
        #data = np.random.randint(0, 21, size=(100, 100))
        #data = np.repeat(np.arange(21).reshape(-1, 1), 21, axis=1)

        plt.figure(figsize=(10, 10))
        # Plot using the PASCAL VOC color palette
        plt.imshow(nda_2DXX_ImgIn, cmap=plt.cm.colors.ListedColormap(colors))
        #plt.colorbar(ticks=np.arange(21))



        # Define legend data
        legend_data = [
            [i, colors[i], class_names[i]]
            for i in range(len(colors))
        ]

        # Create legend handles and labels
        handles = [
            plt.Rectangle((0, 0), 1, 1, color=c)
            for k, c, n in legend_data
        ]
        labels = [n for k, c, n in legend_data]
        
        lst_uniques=np.unique(nda_2DXX_ImgIn).tolist()
        lst_str_uniquesNames=[]
        for lst_uniques_i in lst_uniques:
            print("lst_uniques_i", lst_uniques_i)
            print("class_names[lst_uniques_i]", class_names[lst_uniques_i])
            lst_str_uniquesNames.append(class_names[lst_uniques_i])
        
        # Display legend
        plt.legend(handles=handles, labels=labels, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
        plt.title(str(lst_uniques)+str(lst_str_uniquesNames)+str_imgIn+str(flo_miouIn))
        plt.show()
        
        


In [26]:
#array_1d = np.array([1, 2, 3, 4, 5])
#type(np.unique(array_1d))

# class_DeepLab_Test

In [27]:
class class_DeepLab_Test():#Entrenar modelo
    def mein(self,
            str_dataset2,
            str_entrenaroCargar12,
            str_dirModelLoad12,
            boo_activarCRFIn1
            ):
    
        
        """Se carga el modelo (o entrena), y luego se le pasa una imagen dada por str_filename 
        para que obtenga el miou de deeplab, miou con CRF y plot. La config del CRF esta fija
        es mas que nada solo por cumplir y ponerlo en el informe de que esta funcionando 
        deeplabV3+ mas CRF.
        Esta pensada para la figura Objective 2: Implement state-of-the-art methods, que se tiene
        la imagen procesada por deeplab y gracias a esta funcion la imagen de deeplab+crf (con parametros fijos, sin optimizar de ninguna manera
        """
        
        #str_entrenaroCargar1="Train"
        #str_entrenaroCargar1=str_entrenaroCargar12 #20230613
        #str_dirModelLoad1=r"20230119_1932class_DeepLabembrionB3NC2Tra900Val100Tes100Epo100"
        #str_dirModelLoad1=str_dirModelLoad12
        boo_guardarModeloIn1=True#20230610    
        int_batch_sizeIn1=3#parece que no se puede 4 el monstruito
        int_epocasIn1=10000 #Tiene early stopping
        #str_dataset="embrion"
        #str_dataset=str_dataset2 #20230613
       
        
        

        
        
        #if (str_dataset=="embrion"):
        if (str_dataset2=="embrion"):
            int_image_sizeIn1=1024#20230613
            int_num_classesIn1=2
            #data_dirIn1=r"./COLAB_instance-level-human-parsing2/embryoDataset"
            str_data_dirIn1=r"COLAB_instance-level-human-parsing2/embryoDataset" #PascalTrainVal
            #data_dirIn1=r"PascalTrainVal" #PascalTrainVal
            str_mask_folderIn1=r"EVLstack_SeqDA" #MasksIgnoreClass21
            str_raw_folderIn1=r"video_animal2_proyeccion_SeqDA2" #Raws
            #raw_folderIn1=r"Raws" #Raws
            int_num_train_imagesIn1=105#900
            int_num_val_imagesIn1=10#100
            int_num_test_imagesIn1=30#100 
            str_extensionRaw=r".png"
            num_classes1=2
            str_filename=r"1OR0006"#r"UDF0019"#Esta es de test. TIENE QUE SER DEL CONJUNTO DE TEST... CORRER DEEPLAB CON 1 SOLA EPOCH Y AHI SALEN PRINTEADAS LAS IMAGENES DE CADA CONJUNTO
            tpl_ParametroXY1=(1,1)
            tpl_ParametroRGB1=(1,1,1)
            int_pairwisebilateralCompat1=3
            int_inference1=7
            str_PairwiseoKernel1="GaussianoSmoothnessKernelYBilateraloAppearanceKernel"
            
            
        if (str_dataset2=="embrion_sinDA"):
            int_image_sizeIn1=1024#20230613
            int_num_classesIn1=2
            #data_dirIn1=r"./COLAB_instance-level-human-parsing2/embryoDataset"
            str_data_dirIn1=r"embryoDataset_SinDA" #PascalTrainVal
            #data_dirIn1=r"PascalTrainVal" #PascalTrainVal
            str_mask_folderIn1=r"EVLstack_Seq" #MasksIgnoreClass21
            str_raw_folderIn1=r"video_animal2_proyeccion_Seq" #Raws
            #raw_folderIn1=r"Raws" #Raws
            int_num_train_imagesIn1=100
            int_num_val_imagesIn1=10
            int_num_test_imagesIn1=30 
            str_extensionRaw=r".png"
            num_classes1=2
            str_filename=r"1OR0111"#TIENE QUE SER DEL CONJUNTO DE TEST... CORRER DEEPLAB CON 1 SOLA EPOCH Y AHI SALEN PRINTEADAS LAS IMAGENES DE CADA CONJUNTO
            tpl_ParametroXY1=(1,1)
            tpl_ParametroRGB1=(1,1,1)
            int_pairwisebilateralCompat1=3
            int_inference1=7
            str_PairwiseoKernel1="GaussianoSmoothnessKernelYBilateraloAppearanceKernel"
            

            #mask_folderIn1=r"MasksIgnoreClass21" #MasksIgnoreClass21
        if (str_dataset2=="pascal"):  
            int_image_sizeIn1=512#20230613 OJO ESTO QUIZAS LO HACE MEJORAR. El original deeplabv3+ es con 512 de size
            int_num_classesIn1=22# Con 21 el loss es nan y 0 background
            #data_dirIn1=r"./COLAB_instance-level-human-parsing2/embryoDataset"
            #data_dirIn1=r"COLAB_instance-level-human-parsing2/embryoDataset" #PascalTrainVal
            str_data_dirIn1=r"PascalTrainVal" #PascalTrainVal
            #raw_folderIn1=r"video_animal2_proyeccion_SeqDA2" #Raws
            str_mask_folderIn1=r"MasksIgnoreClass21" #MasksIgnoreClass21
            str_raw_folderIn1=r"Raws" #Raws
            #mask_folderIn1=r"EVLstack_SeqDA" #MasksIgnoreClass21
            int_num_train_imagesIn1=9000
            int_num_val_imagesIn1=2000
            int_num_test_imagesIn1=1000 
            str_extensionRaw=r".jpg"
            num_classes1=22
            str_filename=r"2011_001253"#El caballito. TIENE QUE SER DEL CONJUNTO DE TEST... CORRER DEEPLAB CON 1 SOLA EPOCH Y AHI SALEN PRINTEADAS LAS IMAGENES DE CADA CONJUNTO
            tpl_ParametroXY1=(1,1)
            tpl_ParametroRGB1=(1,1,1)
            int_pairwisebilateralCompat1=3
            int_inference1=7
            str_PairwiseoKernel1="GaussianoSmoothnessKernelYBilateraloAppearanceKernel"
            
        
        if (str_dataset2=="neuron"):  
            int_image_sizeIn1=512#20230613 OJO ESTO QUIZAS LO HACE MEJORAR. El original deeplabv3+ es con 512 de size
            int_num_classesIn1=2# Con 21 el loss es nan y 0 background
            #data_dirIn1=r"./COLAB_instance-level-human-parsing2/embryoDataset"
            #data_dirIn1=r"COLAB_instance-level-human-parsing2/embryoDataset" #PascalTrainVal
            str_data_dirIn1=r"2D_Neuronal_dataset_Proc" #PascalTrainVal
            #raw_folderIn1=r"video_animal2_proyeccion_SeqDA2" #Raws
            str_mask_folderIn1=r"Masks" #MasksIgnoreClass21
            str_raw_folderIn1=r"Raws" #Raws
            #mask_folderIn1=r"EVLstack_SeqDA" #MasksIgnoreClass21
            int_num_train_imagesIn1=150#1400 Despues considera las de data augmentation
            int_num_val_imagesIn1=30#100 Despues considera las de data augmentation
            int_num_test_imagesIn1=30#100 Despues considera las de data augmentation
            str_extensionRaw=r".png"
            num_classes1=2
            str_filename=r"1OR0111"#r"B2_8_fh"#una neuronita. TIENE QUE SER DEL CONJUNTO DE TEST... CORRER DEEPLAB CON 1 SOLA EPOCH Y AHI SALEN PRINTEADAS LAS IMAGENES DE CADA CONJUNTO
            tpl_ParametroXY1=(1,1)
            tpl_ParametroRGB1=(1,1,1)
            int_pairwisebilateralCompat1=3
            int_inference1=7
            str_PairwiseoKernel1="GaussianoSmoothnessKernelYBilateraloAppearanceKernel"
            
        if (str_dataset2=="tissue"):  
            int_image_sizeIn1=512#20230613 OJO ESTO QUIZAS LO HACE MEJORAR. El original deeplabv3+ es con 512 de size
            int_num_classesIn1=2# Con 21 el loss es nan y 0 background
            #data_dirIn1=r"./COLAB_instance-level-human-parsing2/embryoDataset"
            #data_dirIn1=r"COLAB_instance-level-human-parsing2/embryoDataset" #PascalTrainVal
            str_data_dirIn1=r"Warwick_QU_Dataset_Released 2016_07_08_Proc" #PascalTrainVal
            #raw_folderIn1=r"video_animal2_proyeccion_SeqDA2" #Raws
            str_mask_folderIn1=r"Masks" #MasksIgnoreClass21
            str_raw_folderIn1=r"Raws" #Raws
            #mask_folderIn1=r"EVLstack_SeqDA" #MasksIgnoreClass21
            int_num_train_imagesIn1=115#1400 Despues considera las de data augmentation
            int_num_val_imagesIn1=25#100 Despues considera las de data augmentation
            int_num_test_imagesIn1=25#100 Despues considera las de data augmentation
            str_extensionRaw=r".png"
            num_classes1=2
            str_filename=r"1OR0111"#r"B2_8_fh"#una neuronita. TIENE QUE SER DEL CONJUNTO DE TEST... CORRER DEEPLAB CON 1 SOLA EPOCH Y AHI SALEN PRINTEADAS LAS IMAGENES DE CADA CONJUNTO
            tpl_ParametroXY1=(1,1)
            tpl_ParametroRGB1=(1,1,1)
            int_pairwisebilateralCompat1=3
            int_inference1=7
            str_PairwiseoKernel1="GaussianoSmoothnessKernelYBilateraloAppearanceKernel"


        objeto_class_DeepLab=class_DeepLab()
        objeto_class_DeepLab(str_entrenaroCargar12,
                             str_dirModelLoad12,
                             boo_guardarModeloIn1,
                             int_image_sizeIn=int_image_sizeIn1, 
                             int_batch_sizeIn=int_batch_sizeIn1, 

                             int_num_classesIn=int_num_classesIn1,
                             str_data_dirIn=str_data_dirIn1,
                             str_mask_folderIn=str_mask_folderIn1,
                             str_raw_folderIn=str_raw_folderIn1,                          

                             int_num_train_imagesIn=int_num_train_imagesIn1, 
                             int_num_val_imagesIn=int_num_val_imagesIn1, 
                             int_num_test_imagesIn=int_num_test_imagesIn1, 
                             int_epocasIn=int_epocasIn1,
                             boo_activarCRFIn=boo_activarCRFIn1
                            )
        
        self.objeto_class_DeepLab=objeto_class_DeepLab
        
        
        str_DATADIRMASK=r"./"+str_data_dirIn1+r"/"+str_mask_folderIn1+r"/"+str_filename+r".png"
        str_DATADIRRAW=r"./"+str_data_dirIn1+r"/"+str_raw_folderIn1+r"/"+str_filename+str_extensionRaw
        

        if ("TestearModeloConPredictsaStrPath"=="TestearModeloConPredictsaStrPath"):
            
            
                
            test_masksIM8=self.objeto_class_DeepLab.load_dataVBS([str_DATADIRMASK], 
                                                                    mask=True, 
                                                                    to_tensor=False,
                                                                 img_Size1=int_image_sizeIn1#self.int_IMAGE_SIZE
                                                                )

            lst_predsConArgmax8=self.objeto_class_DeepLab.calc_predictionsVBSConArgmax([str_DATADIRRAW], 
                                                                                        self.objeto_class_DeepLab.model,
                                                                                        img_Size2=int_image_sizeIn1#self.int_IMAGE_SIZE
                                                                                      )

            lst_predsSinArgmax8=self.objeto_class_DeepLab.calc_predictionsVBSSinArgmax([str_DATADIRRAW], 
                                                                                        self.objeto_class_DeepLab.model,
                                                                                       img_Size2=int_image_sizeIn1#self.int_IMAGE_SIZE                                                                                      
                                                                                      )
            lst_predsSinArgmax8=lst_predsSinArgmax8[0]
            lst_predsSinArgmax8=np.squeeze(lst_predsSinArgmax8)
            
            
            #lst_predsSinArgmax8=lst_predsSinArgmax8[0]

            m0 = tf.keras.metrics.MeanIoU(num_classes=num_classes1)
            #i=0
            #for lst_preds_i in self.lst_preds0: 
            m0.update_state(test_masksIM8, lst_predsConArgmax8)  
            #m0.update_state(test_masksIM[i], self.lst_preds[i]) 
            #    i+=1
            miou0=m0.result().numpy()*100
            #print("\x1b[34m\"Class Deeplab: MIoU solo de la primera imagen\"\x1b[0m")
            #print("MIoU solo de la primera imagen: ",test_images0)
            print("MIoU de ",str_filename, " : ",miou0)
            
            if("CRF"=="CRF"):#Siempre se tira CRF
                if(boo_activarCRFIn1==True):
                    raise Exception("Error porque estarias tirando doble CRF uno en este if y otro entrenando deeplab con boo_activarCRFIn=True. Deja boo_activarCRFIn1=False. De hecho tengo que sacar esta opcion representada en la variable boo_activarCRFIn1")  
                    
                nda_3DXX3_imgRGB = cv2.cvtColor(cv2.imread(str_DATADIRRAW), cv2.COLOR_BGR2RGB)##Asi esta en A20221210_pipeline5clase. LE estoy pasando al metodo CRF, con metodo CRF no he cargado las imagenes Raw con tensorflow, solo con cv2. Para deeplab si que frecuentemente se leen las imagenes con tensorflow y no con cv2
                
                nda_3DXX3_imgRGB = cv2.resize(nda_3DXX3_imgRGB, (lst_predsSinArgmax8.shape[0], lst_predsSinArgmax8.shape[1]))#lst_predsSinArgmax8 tiene dimensiones anchox alto x numero de clases
                nda_2DXX_imgMask=self.objeto_class_DeepLab.load_dataVBS([str_DATADIRMASK], 
                                                    mask=True, 
                                                    to_tensor=False,
                                                    img_Size1=int_image_sizeIn1#self.int_IMAGE_SIZE
                                                    )[0]
                
                nda_2DXX_imgMask = np.squeeze(nda_2DXX_imgMask)
                
                #self.test_masksIM = self.load_dataVBS(lst_test_masks, mask=True, to_tensor=False)
                
                import import_ipynb
                from A20230112_CRF import class_CRF

                print("lst_predsSinArgmax8.shape: ",lst_predsSinArgmax8.shape)
                print("np.unique(lst_predsSinArgmax8): ",np.unique(lst_predsSinArgmax8))
                
                
                nda_3DXX2_Q = class_CRF().func_aplicar_CRF(nda_2DXX_sdXIn=-1,
                                                 nda_2DXX_sdYIn=-1,
                                                 nda_3DXX2_probmapIn=lst_predsSinArgmax8, 
                                                 tpl_ParametroXYIn=tpl_ParametroXY1, #2 para caballido1 1es el que mas se acerca (1,1) (1,1,1)
                                                 tpl_ParametroRGBIn=tpl_ParametroRGB1, #20 para caballito10 1
                                                 nda_3DXX3_imgRawIn=nda_3DXX3_imgRGB, 
                                                 int_inferenceIn=int_inference1, 
                                                 int_pairwisebilateralCompatIn=int_pairwisebilateralCompat1,
                                                 str_PairwiseoKernel=str_PairwiseoKernel1)
                print("nda_3DXX2_Q.shape: ",nda_3DXX2_Q.shape)
                nda_2DXX_ProbCRFmap_Argmax = np.argmax(nda_3DXX2_Q, axis=2)
                
                from A20230105_MaskImageModel2mIoU import class_MaskImageModel2mIoU
                print("np.unique(nda_2DXX_imgMask): ", np.unique(nda_2DXX_imgMask))
                print("(nda_2DXX_imgMask).shape: ", (nda_2DXX_imgMask).shape)
                print("np.unique(nda_2DXX_ProbCRFmap_Argmax): ", np.unique(nda_2DXX_ProbCRFmap_Argmax))
                print("(nda_2DXX_ProbCRFmap_Argmax).shape: ", (nda_2DXX_ProbCRFmap_Argmax).shape)
                
                
                flo64_ProbCRFmap_Argmax_miou=class_MaskImageModel2mIoU().calcular_postPrediction_TFmIoU_f7(et_3DXX1_imgMaskIn=nda_2DXX_imgMask, 
                                                                                                           nda_2DXX_imgTestPredictedIn=nda_2DXX_ProbCRFmap_Argmax, 
                                                                                                           int_num_classesIn=num_classes1)
                print("flo64_ProbCRFmap_Argmax_miou "+" con CRF "+str_filename+r" :", flo64_ProbCRFmap_Argmax_miou)
                
                
                if (str_dataset2=="embrion"):
                    self.objeto_class_DeepLab.func_plotearEmbrion(nda_2DXX_ImgIn=nda_2DXX_ProbCRFmap_Argmax,
                                                                 str_imgIn="Embrion "+" con CRF "+str_filename+" ",
                                                                 flo_miouIn=flo64_ProbCRFmap_Argmax_miou
                                                                )
                if (str_dataset2=="neuron"):
                    self.objeto_class_DeepLab.func_plotearEmbrion(nda_2DXX_ImgIn=nda_2DXX_ProbCRFmap_Argmax,#2 clases asi que es lo mismo que usar func_plotearEmbrion
                                                                 str_imgIn="Neuron "+" con CRF "+str_filename+" ",
                                                                 flo_miouIn=flo64_ProbCRFmap_Argmax_miou
                                                                )
                if (str_dataset2=="pascal"):
                    self.objeto_class_DeepLab.func_plotearPascal(nda_2DXX_ImgIn=nda_2DXX_ProbCRFmap_Argmax,
                                                                 str_imgIn="Caballito "+" con CRF "+ str_filename+" ",
                                                                 flo_miouIn=flo64_ProbCRFmap_Argmax_miou
                                                                )


                
                plt.imshow(nda_2DXX_ProbCRFmap_Argmax)
                plt.show()
                    
                print(tpl_ParametroXY1,
                        tpl_ParametroRGB1,                        
                        int_pairwisebilateralCompat1,
                        int_inference1,
                        str_PairwiseoKernel1
                        )


In [28]:
class class_DeepLab_Test_2():#Entrenar modelo
    """Esta cosa es para agregar bloque de CRF a deeplab al final"""
    def mein(self,
            str_dataset2,
            str_entrenaroCargar12,
            str_dirModelLoad12):
            
        #str_entrenaroCargar1="Train"
        #str_entrenaroCargar1=str_entrenaroCargar12 #20230613
        #str_dirModelLoad1=r"20230119_1932class_DeepLabembrionB3NC2Tra900Val100Tes100Epo100"
        #str_dirModelLoad1=str_dirModelLoad12
        boo_guardarModeloIn1=False#20230610    
        int_batch_sizeIn1=3#parece que no se puede 4 el monstruito
        int_epocasIn1=10000 #Tiene early stopping
        #str_dataset="embrion"
        #str_dataset=str_dataset2 #20230613
       
        
        
        
        #if (str_dataset=="embrion"):
        if (str_dataset2=="embrion"):
            int_image_sizeIn1=1024#20230613
            int_num_classesIn1=2
            #data_dirIn1=r"./COLAB_instance-level-human-parsing2/embryoDataset"
            str_data_dirIn1=r"COLAB_instance-level-human-parsing2/embryoDataset" #PascalTrainVal
            #data_dirIn1=r"PascalTrainVal" #PascalTrainVal
            str_mask_folderIn1=r"EVLstack_SeqDA" #MasksIgnoreClass21
            str_raw_folderIn1=r"video_animal2_proyeccion_SeqDA2" #Raws
            #raw_folderIn1=r"Raws" #Raws
            int_num_train_imagesIn1=900
            int_num_val_imagesIn1=100
            int_num_test_imagesIn1=100 
            str_extensionRaw=r".png"
            num_classes1=2
            str_filename=r"UDF0019"#una de test
            #tpl_ParametroXY1=(1,1)
            #tpl_ParametroRGB1=(1,1,1)
            #int_pairwisebilateralCompat1=3
            #int_inference1=7
            #str_PairwiseoKernel1="GaussianoSmoothnessKernelYBilateraloAppearanceKernel"
            
        if (str_dataset2=="neuron"):#Todo este bloque agregado 2024
            int_image_sizeIn1=512#20230613 OJO ESTO QUIZAS LO HACE MEJORAR. El original deeplabv3+ es con 512 de size
            int_num_classesIn1=2# Con 21 el loss es nan y 0 background
            #data_dirIn1=r"./COLAB_instance-level-human-parsing2/embryoDataset"
            #data_dirIn1=r"COLAB_instance-level-human-parsing2/embryoDataset" #PascalTrainVal
            str_data_dirIn1=r"2D_Neuronal_dataset_Proc" #PascalTrainVal
            #raw_folderIn1=r"video_animal2_proyeccion_SeqDA2" #Raws
            str_mask_folderIn1=r"Masks" #MasksIgnoreClass21
            str_raw_folderIn1=r"Raws" #Raws
            #mask_folderIn1=r"EVLstack_SeqDA" #MasksIgnoreClass21
            int_num_train_imagesIn1=150#1400 Despues considera las de data augmentation
            int_num_val_imagesIn1=30#100 Despues considera las de data augmentation
            int_num_test_imagesIn1=30#100 Despues considera las de data augmentation
            str_extensionRaw=r".png"
            num_classes1=2
            str_filename=r"1OR0111"#r"B2_8_fh"#una neuronita. TIENE QUE SER DEL CONJUNTO DE TEST... CORRER DEEPLAB CON 1 SOLA EPOCH Y AHI SALEN PRINTEADAS LAS IMAGENES DE CADA CONJUNTO
            #tpl_ParametroXY1=(1,1)
            #tpl_ParametroRGB1=(1,1,1)
            #int_pairwisebilateralCompat1=3
            #int_inference1=7
            #str_PairwiseoKernel1="GaussianoSmoothnessKernelYBilateraloAppearanceKernel"
        

            #mask_folderIn1=r"MasksIgnoreClass21" #MasksIgnoreClass21
        if (str_dataset2=="pascal"):  
            int_image_sizeIn1=512#20230613 OJO ESTO QUIZAS LO HACE MEJORAR. El original deeplabv3+ es con 512 de size
            int_num_classesIn1=22# Con 21 el loss es nan y 0 background
            #data_dirIn1=r"./COLAB_instance-level-human-parsing2/embryoDataset"
            #data_dirIn1=r"COLAB_instance-level-human-parsing2/embryoDataset" #PascalTrainVal
            str_data_dirIn1=r"PascalTrainVal" #PascalTrainVal
            #raw_folderIn1=r"video_animal2_proyeccion_SeqDA2" #Raws
            str_mask_folderIn1=r"MasksIgnoreClass21" #MasksIgnoreClass21
            str_raw_folderIn1=r"Raws" #Raws
            #mask_folderIn1=r"EVLstack_SeqDA" #MasksIgnoreClass21
            int_num_train_imagesIn1=9000
            int_num_val_imagesIn1=2000
            int_num_test_imagesIn1=1000 
            str_extensionRaw=r".jpg"
            num_classes1=22
            str_filename=r"2011_001253"#El caballito
            #tpl_ParametroXY1=(1,1)
            #tpl_ParametroRGB1=(1,1,1)
            #int_pairwisebilateralCompat1=3
            #int_inference1=7
            #str_PairwiseoKernel1="GaussianoSmoothnessKernelYBilateraloAppearanceKernel"


        objeto_class_DeepLab=class_DeepLab()
        objeto_class_DeepLab(str_entrenaroCargar12,
                             str_dirModelLoad12,
                             boo_guardarModeloIn1,
                             int_image_sizeIn=int_image_sizeIn1, 
                             int_batch_sizeIn=int_batch_sizeIn1, 

                             int_num_classesIn=int_num_classesIn1,
                             str_data_dirIn=str_data_dirIn1,
                             str_mask_folderIn=str_mask_folderIn1,
                             str_raw_folderIn=str_raw_folderIn1,                          

                             int_num_train_imagesIn=int_num_train_imagesIn1, 
                             int_num_val_imagesIn=int_num_val_imagesIn1, 
                             int_num_test_imagesIn=int_num_test_imagesIn1, 
                             int_epocasIn=int_epocasIn1,
                             boo_activarCRFIn=False #O True
                            )
        
        self.objeto_class_DeepLab=objeto_class_DeepLab

In [29]:
class class_DeepLab_Test_3():#Utilizar modelo
    """
    Utilizar modelo de manera simple. Ver que pasaba si predecia una imagen usada en training.. me da valor bajo..que raro
    Solo se carga, no se entrena
    """
    def mein(self):
        
        if("ImagenTest"!="ImagenTest"):
            str_imgMaskPathIn9=r"./COLAB_instance-level-human-parsing2/embryoDataset/EVLstack_SeqDA/UDF0067.png"#1OR0060, 1OR0079, 1OR0082 da muy buen miou
            str_imgRawPathIn9=r"./COLAB_instance-level-human-parsing2/embryoDataset/video_animal2_proyeccion_SeqDA2/UDF0067.png"
            print("Dara esto de miou: 79.85447645187378")
        if("ImagenTrain"=="ImagenTrain"):
            str_imgMaskPathIn9=r"./COLAB_instance-level-human-parsing2/embryoDataset/EVLstack_SeqDA/1OR0001.png"#
            str_imgRawPathIn9=r"./COLAB_instance-level-human-parsing2/embryoDataset/video_animal2_proyeccion_SeqDA2/1OR0001.png"
            print("Dara esto de miou: 63.07350397109985")
 
        ###MASCARA
        image = tf.io.read_file(str_imgMaskPathIn9)
        image = tf.image.decode_png(image, channels=1)
        image.set_shape([None, None, 1])
        #image = tf.image.resize(images=image, size=[1024,1024], method="nearest")#20230610

        nda_2DXX_imgMask = image.numpy()#(1024, 1024, 1)
        nda_2DXX_imgMask = np.squeeze(nda_2DXX_imgMask)
        #nda_2DXX_imgMask = nda_2DXX_imgMask.astype(int)
        #print("nda_2DXX_imgMask.shape: ", nda_2DXX_imgMask.shape)
        print("np.max(nda_2DXX_imgMask): ", np.max(nda_2DXX_imgMask))
        print("np.min(nda_2DXX_imgMask): ", np.min(nda_2DXX_imgMask))


        ###RAW
        image = tf.io.read_file(str_imgRawPathIn9)
        image = tf.image.decode_png(image, channels=3)
        image.set_shape([None, None, 3])
        #image = tf.image.resize(images=image, size=[1024,1024])#20230610

        nda_3DXX3_imgRGB = image.numpy()#(1024, 1024, 3)
        #nda_3DXX3_imgRGB = nda_3DXX3_imgRGB.astype(int)
        #print("np.unique(nda_3DXX3_imgRGB): ",np.unique(nda_3DXX3_imgRGB))
        #print("nda_3DXX3_imgRGB.shape: ", nda_3DXX3_imgRGB.shape)
        print("np.max(nda_3DXX3_imgRGB): ", np.max(nda_3DXX3_imgRGB))
        print("np.min(nda_3DXX3_imgRGB): ", np.min(nda_3DXX3_imgRGB))
        
        str_dirModelLoadIn9=r"20230709_0917_A20220911_class_DeepLabVBS_DatasetEmbrionBa3NC2Tr105Va10Te30Ep29D0_6Mi75_8"
        Func_model=tf.keras.models.load_model(r"saved_models/"+str_dirModelLoadIn9) #20230119_1734
        #print("type(nda_3DXX3_imgRGB_disc)", type(nda_3DXX3_imgRGB_disc))
        #print("(nda_3DXX3_imgRGB_disc).shape", (nda_3DXX3_imgRGB_disc).shape)

        #nda_3DXX3_imgRGB_forPredict = nda_3DXX3_imgRGB_disc / 127.5 - 1#ESTA LINEA ES ABSOLUTAMENTE NECESARIA. ANTES DE PREDECIR SE DEBE NORMALIZAR LA IMAGEN
        #nda_3DXX3_imgRGB_forPredict = nda_3DXX3_imgRGB / 127.5#sI PONGO nda_3DXX3_imgRGB, EL PREDICT NO ME TIRA NADA

        nda_3DXX2_probmapDL = Func_model.predict(np.expand_dims((nda_3DXX3_imgRGB / 127.5 - 1), axis=0))
        nda_3DXX2_probmapDL = np.squeeze(nda_3DXX2_probmapDL)

        nda_3DXX2_probmapDL_Argmax = np.argmax(nda_3DXX2_probmapDL, axis=2)

        nda_3DXX2_ProbDLmapIn9=nda_3DXX2_probmapDL.copy()#(1024, 1024, 2)
        nda_2DXX_ProbDLmap_ArgmaxIn9=nda_3DXX2_probmapDL_Argmax.copy()#(1024, 1024)
        
        import import_ipynb
        from A20230105_MaskImageModel2mIoU import class_MaskImageModel2mIoU
        flo64_ProbDLmap_Argmax_miou_Borrame=class_MaskImageModel2mIoU().calcular_postPrediction_TFmIoU_f7(et_3DXX1_imgMaskIn=nda_2DXX_imgMask, 
                                                                                nda_2DXX_imgTestPredictedIn=nda_2DXX_ProbDLmap_ArgmaxIn9, 
                                                                                int_num_classesIn=2)
        print("flo64_ProbDLmap_Argmax_miou CARGANDO MODELO Y HACIENDO TODO SIN CLASE DEEPLAB: ", flo64_ProbDLmap_Argmax_miou_Borrame)


# TESTS

In [ ]:
if __name__ == "__main__":
    
    
    #int_Seleccion="a"
    int_Seleccion=2#2,4
    
    if (int_Seleccion==97):
        str_dirModelLoad122=r"?"
        objeto_class_DeepLab_Test=class_DeepLab_Test()
        objeto_class_DeepLab_Test.mein(str_dataset2="tissue",
                                       str_entrenaroCargar12="Train",#Para entrenar se cambia este por train
                                       str_dirModelLoad12=str_dirModelLoad122,
                                       boo_activarCRFIn1=False
                                      )
    
    
    if (int_Seleccion==98):
        str_dirModelLoad122=r"?"
        objeto_class_DeepLab_Test=class_DeepLab_Test()
        objeto_class_DeepLab_Test.mein(str_dataset2="neuron",
                                       str_entrenaroCargar12="Train",#Para entrenar se cambia este por train
                                       str_dirModelLoad12=str_dirModelLoad122,
                                       boo_activarCRFIn1=False
                                      )
    
    if (int_Seleccion==99):
        str_dirModelLoad122=r"20230709_0917_A20220911_class_DeepLabVBS_DatasetEmbrionBa3NC2Tr105Va10Te30Ep29D0_6Mi75_8"
        objeto_class_DeepLab_Test=class_DeepLab_Test()
        objeto_class_DeepLab_Test.mein(str_dataset2="embrion",
                                       str_entrenaroCargar12="Load",#Para entrenar se cambia este por train
                                       str_dirModelLoad12=str_dirModelLoad122,
                                       boo_activarCRFIn1=False
                                      )
    
    
    if (int_Seleccion==0):
        str_dirModelLoad122=r"20230703_1510_A20220911_class_DeepLabVBS_DatasetNeuronBa3NC2Tr1400Va100Te100Ep44D0_4Mi77_29"
        objeto_class_DeepLab_Test=class_DeepLab_Test()
        objeto_class_DeepLab_Test.mein(str_dataset2="neuron",
                                       str_entrenaroCargar12="Load",#Para entrenar se cambia este por train
                                       str_dirModelLoad12=str_dirModelLoad122,
                                       boo_activarCRFIn1=False
                                      )
    
    elif (int_Seleccion==1):
        str_dirModelLoad122=r"20230613_0947_A20220911_class_DeepLabVBS_embryoDatasetMi79_2Ba3NC2Tr900Va100Te100Ep34D0_8"
        objeto_class_DeepLab_Test=class_DeepLab_Test()
        objeto_class_DeepLab_Test.mein(str_dataset2="embrion",
                                       str_entrenaroCargar12="Train",
                                       str_dirModelLoad12=str_dirModelLoad122,
                                       boo_activarCRFIn1=False
                                      )# se usa UDF0019. UDF0019 (imagen test): 80.00746369361877 con modelo entrenado. 1OR0001(imagen training): 
        
    elif (int_Seleccion==1_2):
        str_dirModelLoad122=r"20230613_0947_A20220911_class_DeepLabVBS_embryoDatasetMi79_2Ba3NC2Tr900Va100Te100Ep34D0_8"
        objeto_class_DeepLab_Test=class_DeepLab_Test()
        objeto_class_DeepLab_Test.mein(str_dataset2="embrion_sinDA",
                                       str_entrenaroCargar12="Load",
                                       str_dirModelLoad12=str_dirModelLoad122,
                                       boo_activarCRFIn1=False
                                      )
    
    elif (int_Seleccion==2):
        #str_dirModelLoad122=r"20230613_0221_A20220911_class_DeepLabVBS_PascalTrainValB3NC22Tra9000Val2000Tes1000Epo10000"
        str_dirModelLoad122=r"20230615_1507_A20220911_class_DeepLabVBS_DatasetPascalBa3NC22Tr9000Va2000Te1000Ep26D1_7Mi52_92"
        objeto_class_DeepLab_Test=class_DeepLab_Test()
        objeto_class_DeepLab_Test.mein(str_dataset2="pascal",
                                        str_entrenaroCargar12="Load",#Cargó bien embrión con mismos resultados
                                        str_dirModelLoad12=str_dirModelLoad122,
                                        boo_activarCRFIn1=False
                                        )
        
    #####ESTOS DE ABAJOS SON PARA TESTEAR AGREGANDOLE EL BLOQUE CRF
    elif (int_Seleccion==3):
        str_dirModelLoad122=r"20230613_0947_A20220911_class_DeepLabVBS_embryoDatasetMi79_2Ba3NC2Tr900Va100Te100Ep34D0_8"
        objeto_class_DeepLab_Test=class_DeepLab_Test_2()
        objeto_class_DeepLab_Test.mein(str_dataset2="embrion",
                                       str_entrenaroCargar12="Load",
                                       str_dirModelLoad12=str_dirModelLoad122
                                      )        
    elif (int_Seleccion==5):#Agregado2023
        str_dirModelLoad122=r"20230801_0011_A20220911_class_DeepLabVBS_DatasetNeuronBa3NC2Tr150Va30Te30Ep42D0_3Mi76_01"
        objeto_class_DeepLab_Test=class_DeepLab_Test_2()
        objeto_class_DeepLab_Test.mein(str_dataset2="neuron",
                                       str_entrenaroCargar12="Load",
                                       str_dirModelLoad12=str_dirModelLoad122
                                      )
    elif (int_Seleccion==4):
        str_dirModelLoad122=r"20230615_1507_A20220911_class_DeepLabVBS_DatasetPascalBa3NC22Tr9000Va2000Te1000Ep26D1_7Mi52_92"
        objeto_class_DeepLab_Test=class_DeepLab_Test_2()
        objeto_class_DeepLab_Test.mein(str_dataset2="pascal",
                                str_entrenaroCargar12="Load",#Cargó bien embrión con mismos resultados
                                str_dirModelLoad12=str_dirModelLoad122
                                )
        
    elif(int_Seleccion=="a"):
        objeto_class_DeepLab_Test3=class_DeepLab_Test_3()
        objeto_class_DeepLab_Test3.mein()

deeplab str_data_dirIn:  PascalTrainVal


2024-02-08 18:34:30.720228: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-08 18:34:30.740056: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-08 18:34:30.740255: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-08 18:34:30.740732: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

"Printear Datasets"
Train Dataset: <BatchDataset element_spec=(TensorSpec(shape=(3, 512, 512, 3), dtype=tf.float32, name=None), TensorSpec(shape=(3, 512, 512, 1), dtype=tf.uint8, name=None))>
Val Dataset: <BatchDataset element_spec=(TensorSpec(shape=(3, 512, 512, 3), dtype=tf.float32, name=None), TensorSpec(shape=(3, 512, 512, 1), dtype=tf.uint8, name=None))>
Test Dataset: <BatchDataset element_spec=(TensorSpec(shape=(3, 512, 512, 3), dtype=tf.float32, name=None), TensorSpec(shape=(3, 512, 512, 1), dtype=tf.uint8, name=None))>
len(train_images) 9000
len(val_images) 2000
len(test_images) 1000
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
set_common_strings1 Train Val:  set()
set_common_strings2 Train Test:  set()
set_common_strings3 Val Test:  set()
SE METIO A LOAD
Modelo a cargar:  20230615_1507_A20220911_class_DeepLabVBS_DatasetPascalBa3NC22Tr9000Va2000Te1000Ep26D1_7Mi52_92
Modelo ca

In [ ]:
#0/0

In [ ]:
if False:
    import numpy as np

    my_list = [1, 2, 3, 4, 5]
    sample = np.random.choice(my_list, size=3, replace=False)
    print(sample)  # Output: a random sample of 3 elements from my_list


In [ ]:
#objeto_class_DeepLab_Test.objeto_class_DeepLab.model

In [ ]:
if False:
    test_masksIM8=objeto_class_DeepLab_Test.objeto_class_DeepLab.load_dataVBS([r'./COLAB_instance-level-human-parsing2/embryoDataset/EVLstack_SeqDA/UDF0086.png'], 
                                                                                mask=True, 
                                                                                to_tensor=False)

    lst_predsConArgmax8=objeto_class_DeepLab_Test.objeto_class_DeepLab.calc_predictionsVBSConArgmax([r'./COLAB_instance-level-human-parsing2/embryoDataset/video_animal2_proyeccion_SeqDA2/UDF0086.png'], 
                                                                                                      objeto_class_DeepLab_Test.objeto_class_DeepLab.model)

    #lst_predsSinArgmax8=objeto_class_DeepLab_Test.objeto_class_DeepLab.calc_predictionsVBSSinArgmax([r'./COLAB_instance-level-human-parsing2/embryoDataset/video_animal2_proyeccion_SeqDA2/UDF0085.png'], 
    #                                                                                                  objeto_class_DeepLab_Test.objeto_class_DeepLab.model)
    #lst_predsSinArgmax8=lst_predsSinArgmax8[0]

    m0 = tf.keras.metrics.MeanIoU(num_classes=2)
    #i=0
    #for lst_preds_i in self.lst_preds0: 
    m0.update_state(test_masksIM8, lst_predsConArgmax8)  
        #m0.update_state(test_masksIM[i], self.lst_preds[i]) 
    #    i+=1
    miou0=m0.result().numpy()*100
    print("\x1b[34m\"Class Deeplab: MIoU solo de la primera imagen\"\x1b[0m")
    #print("MIoU solo de la primera imagen: ",test_images0)
    print("MIoU solo de la primera imagen: ",miou0)

In [ ]:
#lst_predsConArgmax8.shape

In [ ]:
if False:
    if("PrimeraImagenMioU"=="PrimeraImagenMioU"):
        self.test_masksIM0 = self.load_dataVBS(test_masks0, mask=True, to_tensor=False, img_Size1=self.int_IMAGE_SIZE)
        self.lst_predsConArgmax0=self.calc_predictionsVBSConArgmax(test_images0, self.model) 
        self.lst_predsSinArgmax0=self.calc_predictionsVBSSinArgmax(test_images0, self.model)#es una lista por eso la modifico abajo     
        self.lst_predsSinArgmax0=self.lst_predsSinArgmax0[0]

        m0 = tf.keras.metrics.MeanIoU(num_classes=int_num_classesIn)
        #i=0
        #for lst_preds_i in self.lst_preds0: 
        m0.update_state(self.test_masksIM0, self.lst_predsConArgmax0)  
            #m0.update_state(test_masksIM[i], self.lst_preds[i]) 
        #    i+=1
        self.miou0=m0.result().numpy()*100
        print("\x1b[34m\"Class Deeplab: MIoU solo de la primera imagen\"\x1b[0m")
        print("MIoU solo de la primera imagen: ",test_images0)
        print("MIoU solo de la primera imagen: ",self.miou0)

In [ ]:
if False:    
    if __name__ == "__main__":


        str_entrenaroCargar1="Train"
        str_dirModelLoad1=r"20230119_1932class_DeepLabembrionB3NC2Tra900Val100Tes100Epo100"
        boo_guardarModeloIn1=True#20230610    
        int_batch_sizeIn1=3#parece que no se puede 4 el monstruito
        int_epocasIn1=1#Tiene early stopping
        str_dataset="embrion"

        if (str_dataset=="embrion"):
            int_image_sizeIn1=1024#20230613
            int_num_classesIn1=2
            #data_dirIn1=r"./COLAB_instance-level-human-parsing2/embryoDataset"
            str_data_dirIn1=r"COLAB_instance-level-human-parsing2/embryoDataset" #PascalTrainVal
            #data_dirIn1=r"PascalTrainVal" #PascalTrainVal
            str_mask_folderIn1=r"EVLstack_SeqDA" #MasksIgnoreClass21
            str_raw_folderIn1=r"video_animal2_proyeccion_SeqDA2" #Raws
            #raw_folderIn1=r"Raws" #Raws
            int_num_train_imagesIn1=900
            int_num_val_imagesIn1=100
            int_num_test_imagesIn1=100 

            #mask_folderIn1=r"MasksIgnoreClass21" #MasksIgnoreClass21
        if (str_dataset=="pascal"):  
            int_image_sizeIn1=512#20230613
            int_num_classesIn1=22# Con 21 el loss es nan y 0 background
            #data_dirIn1=r"./COLAB_instance-level-human-parsing2/embryoDataset"
            #data_dirIn1=r"COLAB_instance-level-human-parsing2/embryoDataset" #PascalTrainVal
            str_data_dirIn1=r"PascalTrainVal" #PascalTrainVal
            #raw_folderIn1=r"video_animal2_proyeccion_SeqDA2" #Raws
            str_mask_folderIn1=r"MasksIgnoreClass21" #MasksIgnoreClass21
            str_raw_folderIn1=r"Raws" #Raws
            #mask_folderIn1=r"EVLstack_SeqDA" #MasksIgnoreClass21
            int_num_train_imagesIn1=9000
            int_num_val_imagesIn1=2000
            int_num_test_imagesIn1=1000 


        objeto_class_DeepLab=class_DeepLab()
        objeto_class_DeepLab(str_entrenaroCargar1,
                             str_dirModelLoad1,
                             boo_guardarModeloIn1,
                             int_image_sizeIn=int_image_sizeIn1, 
                             int_batch_sizeIn=int_batch_sizeIn1, 

                             int_num_classesIn=int_num_classesIn1,
                             str_data_dirIn=str_data_dirIn1,
                             str_mask_folderIn=str_mask_folderIn1,
                             str_raw_folderIn=str_raw_folderIn1,                          

                             int_num_train_imagesIn=int_num_train_imagesIn1, 
                             int_num_val_imagesIn=int_num_val_imagesIn1, 
                             int_num_test_imagesIn=int_num_test_imagesIn1, 
                             int_epocasIn=int_epocasIn1)


In [ ]:
if False:    
    import os
    import glob

    str_directory = "./"
    set_extensions = set()
    for file_path in glob.glob(os.path.join(str_directory, "*")):
        if os.path.isfile(file_path):
            file_extension = os.path.splitext(file_path)[1]
            set_extensions.add(file_extension)
    lst_extensions=list(set_extensions)
    lst_extensions[0]
    #print("File extensions in the directory:")
    #for extension in extensions:
    #    print(extension)

In [ ]:
if False:    
    import time

    flo_start_time = time.time()

    #history = self.model.fit(train_dataset, validation_data=val_dataset, epochs=int_EPOCAS, callbacks=[callback])
    time.sleep(3)
    flo_end_time = time.time()

    flo_duration_seconds = flo_end_time - flo_start_time
    flo_duration_hours = flo_duration_seconds / 3600  # Convert seconds to hours

    #str_formatted_duration = "{:.2f} hours".format(flo_duration_hours)
    #str_formatted_duration = round(flo_duration_hours, 1).replace(".", "_")
    flo_rounded_miou = round(flo_duration_hours, 1)
    str_rounded_miou = str(flo_rounded_miou).replace(".", "_")

    print("Training duration:", str_rounded_miou)


In [ ]:
if False: #Usar este en el futuro si es que se quieren tener sets random (no sorted) de training, validation y test    
    import os
    import random

    mask_folder = r"embryoDataset_SinDA/EVLstack_Seq"
    raw_folder = r"embryoDataset_SinDA/video_animal2_proyeccion_Seq"

    test_ratio = 0.2  # Percentage of files for the test set
    validation_ratio = 0.1  # Percentage of files for the validation set

    seed = 123  # Seed value for reproducibility

    # Set the seed value
    random.seed(seed)

    # Get the list of file names in the raw folder
    file_names = os.listdir(raw_folder)

    # Shuffle the file names randomly
    random.shuffle(file_names)

    # Calculate the number of files for each set
    total_files = len(file_names)
    test_count = int(total_files * test_ratio)
    validation_count = int(total_files * validation_ratio)
    training_count = total_files - test_count - validation_count

    # Split the file names into sets
    test_set = random.sample(file_names, test_count)
    validation_set = random.sample(set(file_names) - set(test_set), validation_count)
    training_set = list(set(file_names) - set(test_set) - set(validation_set))

    # Create the full file paths for each set
    test_set_files = [(os.path.join(raw_folder, file), os.path.join(mask_folder, file)) for file in test_set]
    validation_set_files = [(os.path.join(raw_folder, file), os.path.join(mask_folder, file)) for file in validation_set]
    training_set_files = [(os.path.join(raw_folder, file), os.path.join(mask_folder, file)) for file in training_set]

    # Print the file lists
    print("Test Set:")
    print(test_set_files)
    print("Validation Set:")
    print(validation_set_files)
    print("Training Set:")
    print(training_set_files)


In [ ]:
if False:    
    import os
    import random

    mask_folder = r"embryoDataset_SinDA/EVLstack_Seq"
    raw_folder = r"embryoDataset_SinDA/video_animal2_proyeccion_Seq"

    test_ratio = 0.2  # Percentage of files for the test set
    validation_ratio = 0.1  # Percentage of files for the validation set

    seed = 123  # Seed value for reproducibility

    # Set the seed value
    random.seed(seed)

    # Get the list of file names in the raw folder
    file_names = os.listdir(raw_folder)

    # Shuffle the file names randomly
    random.shuffle(file_names)

    # Calculate the number of files for each set
    total_files = len(file_names)
    training_count = int(total_files * (1 - test_ratio - validation_ratio))
    validation_count = int(total_files * validation_ratio)
    test_count = total_files - training_count - validation_count

    # Split the file names into sets
    training_set = file_names[:training_count]
    validation_set = file_names[training_count:training_count + validation_count]
    test_set = file_names[training_count + validation_count:]

    # Create the full file paths for each set
    training_set_files_Masks = [os.path.join(mask_folder, file) for file in training_set]
    training_set_files_Raws = [os.path.join(raw_folder, file) for file in training_set]


    validation_set_files_Masks = [os.path.join(mask_folder, file) for file in validation_set]
    validation_set_files_Raws = [os.path.join(raw_folder, file) for file in validation_set]

    test_set_files_Masks = [os.path.join(mask_folder, file) for file in test_set]
    test_set_files_Raws = [os.path.join(raw_folder, file) for file in test_set]


    #validation_set_files = [(os.path.join(raw_folder, file), os.path.join(mask_folder, file)) for file in validation_set]
    #test_set_files = [(os.path.join(raw_folder, file), os.path.join(mask_folder, file)) for file in test_set]

    # Print the file lists
    print("Training Set:")
    for training_set_files_Masks_i in training_set_files_Masks:
        print(training_set_files_Masks_i)
    #print(training_set_files)
    print("Validation Set:")
    for validation_set_files_Masks_i in validation_set_files_Masks:
        print(validation_set_files_Masks_i)
    #print(validation_set_files)
    print("Test Set:")
    for test_set_files_Masks_i in test_set_files_Masks:
        print(test_set_files_Masks_i)
    #print(test_set_files)


In [ ]:
if False:
    import os

    def get_filenames(file_paths):
        return [os.path.splitext(os.path.basename(path))[0] for path in file_paths]

    def compare_filename_lists(list1, list2):
        filenames1 = set(get_filenames(list1))
        filenames2 = set(get_filenames(list2))

        common_filenames = filenames1.intersection(filenames2)

        return common_filenames

    # Example usage
    list1 = training_set_files_Masks
    list2 = training_set_files_Raws

    common_filenames = compare_filename_lists(list1, list2)
    print(common_filenames)


In [ ]:
if False:    
    import os

    def func_MismosFilenames(
                             lst_setMasksIn, 
                             lst_setRawsIn): 

        def get_filenames(lst_str_file_paths):
            return {os.path.splitext(os.path.basename(str_path))[0] for str_path in lst_str_file_paths}

        def compare_filename_lists(lst_str_list1, lst_str_list2):
            set_str_filenames1 = get_filenames(lst_str_list1)
            set_str_filenames2 = get_filenames(lst_str_list2)

            set_str_common_filenames = set_str_filenames1.intersection(set_str_filenames2)
            if (len(set_str_common_filenames) > 0):
                pass
            return (len(set_str_common_filenames) > 0)

        # Example usage
        #list1 = training_set_files_Masks
        #list2 = validation_set_files_Raws

        boo_have_same_filenames = compare_filename_lists(lst_setMasksIn, lst_setRawsIn)
        #print(boo_have_same_filenames)
        return(boo_have_same_filenames)

    boo_SetsTes = func_MismosFilenames(lst_test_set_Masks, lst_validation_set_Raws)#lst_test_set_Raws

    if False:
        boo_SetsVal = func_MismosFilenames(lst_validation_set_Masks, lst_validation_set_Raws)
        boo_SetsTra = func_MismosFilenames(lst_training_set_Masks, lst_training_set_Raws)

        if((boo_SetsTra!=True) or (boo_SetsVal!=True) or (boo_SetsTes!=True)):
            raise Exception("No coinciden los archivos representados en las listas:"+
                            "training_set_files_Masks, training_set_files_Raws"+
                            "validation_set_files_Masks, validation_set_files_Raws"+
                            "test_set_files_Masks, test_set_files_Raws"
                           )  
        else:
            print("Los datasets Masks y Raws de Training, Validation y Test coinciden")


In [ ]:
#{os.path.splitext(os.path.basename(str_path))[0] for str_path in lst_test_set_Masks}

In [ ]:
#{os.path.splitext(os.path.basename(str_path))[0] for str_path in lst_validation_set_Raws}

In [ ]:
#lst_validation_set_Raws

In [ ]:
if False:    
    def compare_filename_lists(list1, list2):
        filenames1 = {os.path.splitext(os.path.basename(file1))[0] for file1 in list1}
        filenames2 = {os.path.splitext(os.path.basename(file2))[0] for file2 in list2}

        return filenames1 == filenames2


    have_same_filenames = compare_filename_lists(lst_test_set_Masks, lst_test_set_Raws)
    #have_same_filenames = compare_filename_lists(lst_validation_set_Masks, lst_validation_set_Raws)
    #have_same_filenames = compare_filename_lists(lst_training_set_Masks, lst_training_set_Raws)

    print(have_same_filenames)


In [ ]:
#lst_test_set_Masks

In [ ]:
#lst_test_set_Raws

In [ ]:
if False:    
    def func_NoDebenIntersectar(list1, list2):
        filenames1 = set(os.path.splitext(os.path.basename(file1))[0] for file1 in list1)
        filenames2 = set(os.path.splitext(os.path.basename(file2))[0] for file2 in list2)

        common_filenames = filenames1.intersection(filenames2)#Se intersectan si o no?
        if (len(common_filenames)>0):
            print("Se intersectan")
        else:
            print("No see intersectan")

        return (len(common_filenames) > 0)

    boo_SetsTes = compare_filename_lists(lst_test_set_Masks, lst_validation_set_Raws)
    boo_SetsVal = compare_filename_lists(lst_validation_set_Masks, lst_validation_set_Raws)
    boo_SetsTra = compare_filename_lists(lst_training_set_Masks, lst_training_set_Raws)


    if((boo_SetsTra!=True) or (boo_SetsVal!=True) or (boo_SetsTes!=True)):
        raise Exception("No coinciden los archivos representados en las listas:"+
                        "training_set_files_Masks, training_set_files_Raws"+
                        "validation_set_files_Masks, validation_set_files_Raws"+
                        "test_set_files_Masks, test_set_files_Raws"
                       )  
    else:
        print("Los datasets Masks y Raws de Training, Validation y Test coinciden")

In [ ]:
if False:
    filenames1 = set(os.path.splitext(os.path.basename(file1))[0] for file1 in lst_validation_set_Raws)
    filenames2 = set(os.path.splitext(os.path.basename(file2))[0] for file2 in lst_test_set_Raws)

    common_filenames = filenames1.intersection(filenames2)#Se intersectan si o no?
    common_filenames

In [ ]:
#len(lst_str_file_names_Raws)

In [ ]:
#len(lst_str_file_names_Path_Masks)
#lst_test_set

In [ ]:
#lst_test_set

In [ ]:
#lst_test_set_Masks

In [ ]:
if False:
    print("oficial")
    def func_comprobarNoTraslape_TrainValTest(lst_str_filenamePath_testSet_MasksIn, lst_str_filenamePath_validationSet_MasksIn, lst_str_filenamePath_trainingSet_MasksIn,
                                              lst_str_filenamePath_testSet_RawsIn, lst_str_filenamePath_validationSet_RawsIn, lst_str_filenamePath_trainingSet_RawsIn
                                              ):

        # Check if there are no intersections between list1 and list2
        boo_intersectionsMasks_12 = len(set(lst_str_filenamePath_testSet_MasksIn).intersection(set(lst_str_filenamePath_validationSet_MasksIn))) > 0

        # Check if there are no intersections between list1 and list3
        boo_intersectionsMasks_13 = len(set(lst_str_filenamePath_testSet_MasksIn).intersection(set(lst_str_filenamePath_trainingSet_MasksIn))) > 0

        # Check if there are no intersections between list2 and list3
        boo_intersectionsMasks_23 = len(set(lst_str_filenamePath_validationSet_MasksIn).intersection(set(lst_str_filenamePath_trainingSet_MasksIn))) > 0

        # Check if there are no intersections between all three lists
        boo_intersectionsMasks_all = boo_intersectionsMasks_12 or boo_intersectionsMasks_13 or boo_intersectionsMasks_23

        # Check if there are no intersections between list1 and list2
        boo_intersectionsRaws_12 = len(set(lst_str_filenamePath_testSet_RawsIn).intersection(set(lst_str_filenamePath_validationSet_RawsIn))) > 0

        # Check if there are no intersections between list1 and list3
        boo_intersectionsRaws_13 = len(set(lst_str_filenamePath_testSet_RawsIn).intersection(set(lst_str_filenamePath_trainingSet_RawsIn))) > 0

        # Check if there are no intersections between list2 and list3
        boo_intersectionsRaws_23 = len(set(lst_str_filenamePath_validationSet_RawsIn).intersection(set(lst_str_filenamePath_trainingSet_RawsIn))) > 0

        # Check if there are no intersections between all three lists
        boo_intersectionsRaws_all = boo_intersectionsRaws_12 or boo_intersectionsRaws_13 or boo_intersectionsRaws_23

        print(r"Intersections between MASKS list1 and list2 ?:", boo_intersectionsMasks_12)
        print(r"Intersections between MASKS list1 and list3 ?:", boo_intersectionsMasks_13)
        print(r"Intersections between MASKS list2 and list3 ?:", boo_intersectionsMasks_23)
        print(r"Intersections between MASKS all three lists ?:", boo_intersectionsMasks_all)

        print(r"Intersections between RAWS list1 and list2 ?:", boo_intersectionsRaws_12)
        print(r"Intersections between RAWS list1 and list3 ?:", boo_intersectionsRaws_13)
        print(r"Intersections between RAWS list2 and list3 ?:", boo_intersectionsRaws_23)
        print(r"Intersections between RAWS all three lists ?:", boo_intersectionsRaws_all)

        if(boo_intersectionsMasks_all!=False or boo_intersectionsRaws_all!=False):
            raise Exception("Por lo menos uno de los conjuntos de TrainValTest se traslapa")
        return("OK. TrainValTest no se traslapan")

    func_comprobarNoTraslape_TrainValTest(lst_str_filenamePath_testSet_Masks, lst_str_filenamePath_validationSet_Masks, lst_str_filenamePath_trainingSet_Masks,
                                          lst_str_filenamePath_testSet_Raws, lst_str_filenamePath_validationSet_Raws, lst_str_filenamePath_trainingSet_Raws
                                          )

In [ ]:
if False:
    print("oficial")
    def func_comprobarMaskyRawsiguales_TrainValTest(lst_str_filenamePath_testSet_MasksIn, lst_str_filenamePath_testSet_RawsIn,
                                                    lst_str_filenamePath_validationSet_MasksIn, lst_str_filenamePath_validationSet_RawsIn,
                                                    lst_str_filenamePath_trainingSet_MasksIn, lst_str_filenamePath_trainingSet_RawsIn,
                                                   ):

        def func_DirectoriosTienenMismosArchivos(list1, list2):
            filenames1 = {os.path.splitext(os.path.basename(file1))[0] for file1 in list1}
            filenames2 = {os.path.splitext(os.path.basename(file2))[0] for file2 in list2}
            boo_SonIguales = filenames1 == filenames2
            return (boo_SonIguales)

        boo_test_MasksRaws = func_DirectoriosTienenMismosArchivos(lst_str_filenamePath_testSet_MasksIn, lst_str_filenamePath_testSet_RawsIn)#lst_str_filenamePath_testSet_Raws
        boo_validation_MasksRaws = func_DirectoriosTienenMismosArchivos(lst_str_filenamePath_validationSet_MasksIn, lst_str_filenamePath_validationSet_RawsIn)
        boo_training_MasksRaws = func_DirectoriosTienenMismosArchivos(lst_str_filenamePath_trainingSet_MasksIn, lst_str_filenamePath_trainingSet_RawsIn)

        if(boo_test_MasksRaws!=True or boo_validation_MasksRaws!=True or boo_training_MasksRaws!=True):
            raise Exception("No tienen los mismos filepaths los conjuntos de MaskyRaw de alguno de los conjuntos de train, val o test")

        return("OK. MaskyRaw tienen los mismos archivos para TrainValTest")
        #print(boo_test_MasksRaws)

    func_comprobarMaskyRawsiguales_TrainValTest(lst_str_filenamePath_testSet_Masks, lst_str_filenamePath_testSet_Raws,
                                                lst_str_filenamePath_validationSet_Masks, lst_str_filenamePath_validationSet_Raws,
                                                lst_str_filenamePath_trainingSet_Masks, lst_str_filenamePath_trainingSet_Raws
                                                )

In [ ]:
if False:
    print("oficial")
    import os
    import random
    from glob import glob

    str_mask_folder = r"COLAB_instance-level-human-parsing2/embryoDataset/EVLstack_SeqDA"
    str_raw_folder = r"COLAB_instance-level-human-parsing2/embryoDataset/video_animal2_proyeccion_SeqDA2"

    #have_same_filenames = func_DirectoriosTienenMismosArchivos(lst_str_file_names_Masks, lst_str_file_names_Raws)
    #if (have_same_filenames!=True):
    #    raise Exception("No tienen los mismos nombres de archivo las carpetas")

    int_testFilescount=30#Numeros sin considerar data augmentation
    int_validationFilescount=10#Numeros sin considerar data augmentation
    int_trainingFilescount=105#Numeros sin considerar data augmentation. Este se va a multpilicar por 8 con DA



    if("datasetembrion"=="datasetembrion"):#Esto es para sacar a los data augmented de test y validation
        if (int_testFilescount+int_validationFilescount+int_trainingFilescount>145):
            raise Exception("Hay solo 145 imagenes, hay que achicar int_testFilescount o int_validationFilescount o int_trainingFilescount")

        seed = 123 
        random.seed(seed)

        lst_str_filenamePath_10R_Masks = glob(os.path.join(str_mask_folder, "1OR*"))
        lst_str_filenamePath_10R_Raws = glob(os.path.join(str_raw_folder, "1OR*"))

        lst_str_filenamePath_Masks = glob(str_mask_folder + '/*')
        lst_str_filenamePath_Raws = glob(str_raw_folder + '/*')

        #Test
        lst_str_filenamePath_10R_testSet = np.random.sample(set(lst_str_filenamePath_10R_Masks), int_testFilescount)#Selecciono solo de entre 1OR que es la imagen sin ninguna rotacion
        lst_str_filename_10R_testSet = [os.path.splitext(os.path.basename(path))[0] for path in lst_str_filenamePath_10R_testSet]#Me quedo con los filenames. Ejemplo: 1OR0027
        lst_str_filenamePath_10R_testSet_Masks = [path for path in lst_str_filenamePath_Masks if os.path.splitext(os.path.basename(path))[0] in lst_str_filename_10R_testSet]#Recupero el path a partir de los filenames para Mask
        lst_str_filenamePath_10R_testSet_Raws = [path for path in lst_str_filenamePath_Raws if os.path.splitext(os.path.basename(path))[0] in lst_str_filename_10R_testSet]#Recupero el path a partir de los filenames para Raws

        #Validation
        lst_str_filenamePath_10R_validationSet = np.random.sample(set(lst_str_filenamePath_10R_Masks)-set(lst_str_filenamePath_10R_testSet_Masks), int_validationFilescount)#Selecciono solo de entre 1OR que es la imagen sin ninguna rotacion
        lst_str_filename_10R_validationSet = [os.path.splitext(os.path.basename(path))[0] for path in lst_str_filenamePath_10R_validationSet]#Me quedo con los filenames    
        lst_str_filenamePath_10R_validationSet_Masks = [path for path in lst_str_filenamePath_Masks if os.path.splitext(os.path.basename(path))[0] in lst_str_filename_10R_validationSet]
        lst_str_filenamePath_10R_validationSet_Raws = [path for path in lst_str_filenamePath_Raws if os.path.splitext(os.path.basename(path))[0] in lst_str_filename_10R_validationSet]

        #Training
        lst_str_filenamePath_10R_trainingSet = np.random.sample(set(lst_str_filenamePath_10R_Masks)-set(lst_str_filenamePath_10R_testSet_Masks)-set(lst_str_filenamePath_10R_validationSet_Masks), int_trainingFilescount) #Selecciono solo de entre 1OR que es la imagen sin ninguna rotacion
        lst_str_filename_10R_trainingSet = [os.path.splitext(file)[0][-4:]  for file in lst_str_filenamePath_10R_trainingSet] # Me quedo solo con el nombre del archivo sin prefijos ni sufijos y con extensión
        lst_str_filenamePath_10R_trainingSet_Masks = [file_path for file_path in lst_str_filenamePath_Masks if any(substr in file_path for substr in lst_str_filename_10R_trainingSet)]
        lst_str_filenamePath_10R_trainingSet_Raws = [file_path for file_path in lst_str_filenamePath_Raws if any(substr in file_path for substr in lst_str_filename_10R_trainingSet)]


        lst_str_filenamePath_testSet_Masks=lst_str_filenamePath_10R_testSet_Masks
        lst_str_filenamePath_testSet_Raws=lst_str_filenamePath_10R_testSet_Raws
        lst_str_filenamePath_validationSet_Masks=lst_str_filenamePath_10R_validationSet_Masks
        lst_str_filenamePath_validationSet_Raws=lst_str_filenamePath_10R_validationSet_Raws
        lst_str_filenamePath_trainingSet_Masks=lst_str_filenamePath_10R_trainingSet_Masks
        lst_str_filenamePath_trainingSet_Raws=lst_str_filenamePath_10R_trainingSet_Raws



    #print(len(lst_training_set_Masks))
    #print(len(lst_validation_set_Masks))
    #print(len(lst_test_set_Masks))

    #print(145*8-840-10*8-30*8)
    print("len(lst_str_filenamePath_testSet_Masks)", len(lst_str_filenamePath_testSet_Masks))
    print("len(lst_str_filenamePath_validationSet_Masks)", len(lst_str_filenamePath_validationSet_Masks))
    print("len(lst_str_filenamePath_trainingSet_Masks)", len(lst_str_filenamePath_trainingSet_Masks))

    print("len(lst_str_filenamePath_testSet_Raws)", len(lst_str_filenamePath_testSet_Raws))
    print("len(lst_str_filenamePath_validationSet_Raws)", len(lst_str_filenamePath_validationSet_Raws))
    print("len(lst_str_filenamePath_trainingSet_Raws)", len(lst_str_filenamePath_trainingSet_Raws))

    if True:

        # Print the file lists
        print("Test Set MASK:")
        for lst_str_filenamePath_testSet_Masks_i in lst_str_filenamePath_testSet_Masks:
            print(lst_str_filenamePath_testSet_Masks_i)
        print("Validation Set MASK:")
        for lst_str_filenamePath_validationSet_Masks_i in lst_str_filenamePath_validationSet_Masks:
            print(lst_str_filenamePath_validationSet_Masks_i)
        print("Training Set MASK:")
        for lst_str_filenamePath_trainingSet_Masks_i in lst_str_filenamePath_trainingSet_Masks:
            print(lst_str_filenamePath_trainingSet_Masks_i)

        print("Test Set RAWS:")
        for lst_str_filenamePath_testSet_Raws_i in lst_str_filenamePath_testSet_Raws:
            print(lst_str_filenamePath_testSet_Raws_i)
        print("Validation Set RAWS:")
        for lst_str_filenamePath_validationSet_Raws_i in lst_str_filenamePath_validationSet_Raws:
            print(lst_str_filenamePath_validationSet_Raws_i)
        print("Training Set RAWS:")
        for lst_str_filenamePath_trainingSet_Raws_i in lst_str_filenamePath_trainingSet_Raws:
            print(lst_str_filenamePath_trainingSet_Raws_i)



In [ ]:
if False:    
    import os
    import random

    mask_folder = r"embryoDataset_SinDA/EVLstack_Seq"
    raw_folder = r"embryoDataset_SinDA/video_animal2_proyeccion_Seq"

    test_ratio = 0.2  # Percentage of files for the test set
    validation_ratio = 0.1  # Percentage of files for the validation set

    seed = 123  # Seed value for reproducibility

    # Set the seed value
    random.seed(seed)

    # Get the list of file names in the raw folder
    file_names = os.listdir(raw_folder)

    # Keep only the last four characters (excluding extension) in the file names
    file_names = [os.path.splitext(file)[0][-4:] + os.path.splitext(file)[1] for file in file_names]

    # Calculate the number of files for each set
    total_files = len(file_names)
    test_count = int(total_files * test_ratio)
    validation_count = int(total_files * validation_ratio)
    training_count = total_files - test_count - validation_count

    # Split the file names into sets
    test_set = random.sample(file_names, test_count)
    validation_set = random.sample(set(file_names) - set(test_set), validation_count)
    training_set = list(set(file_names) - set(test_set) - set(validation_set))

    # Create the full file paths for each set
    test_set_files_Masks = [os.path.join(mask_folder, file) for file in test_set]
    test_set_files_Raws = [os.path.join(raw_folder, file) for file in test_set]

    validation_set_files_Masks = [os.path.join(mask_folder, file) for file in validation_set]
    validation_set_files_Raws = [os.path.join(raw_folder, file) for file in validation_set]

    training_set_files_Masks = [os.path.join(mask_folder, file) for file in file_names if any(filename in file for filename in training_set)]
    training_set_files_Raws = [os.path.join(raw_folder, file) for file in file_names if any(filename in file for filename in training_set)]

    # Print the file lists
    print("Test Set:")
    for test_set_files_Masks_i in test_set_files_Masks:
        print(test_set_files_Masks_i)
    print("Validation Set:")
    for validation_set_files_Masks_i in validation_set_files_Masks:
        print(validation_set_files_Masks_i)
    print("Training Set:")
    for training_set_files_Masks_i in training_set_files_Masks:
        print(training_set_files_Masks_i)


In [ ]:
#type(formatted_duration)

In [ ]:
#pip install tensorflow

In [ ]:
#cv2.imread(r"./COLAB_instance-level-human-parsing2/embryoDataset/video_animal2_proyeccion_SeqDA/UDF0085.jpg", cv2.COLOR_BGR2RGB).shape

In [ ]:
#cv2.imread(r"./COLAB_instance-level-human-parsing2/embryoDataset/EVLstack_SeqDA/UDF0085.png", cv2.COLOR_BGR2RGB).shape

In [ ]:
if False:    
    #if __name__ == "__main__": 
    import os
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

    import tensorflow as tf
    import numpy as np
    import import_ipynb
    from A20230530_TransformarImagenes import class_TransformarImagen as CTI
    import matplotlib.pyplot as plt

    print("TensorFlow version:", tf.__version__)

    str_dirModelLoadIn9=r"20230709_0917_A20220911_class_DeepLabVBS_DatasetEmbrionBa3NC2Tr105Va10Te30Ep29D0_6Mi75_8"

    str_imgMaskPathIn9=r"./COLAB_instance-level-human-parsing2/embryoDataset/EVLstack_SeqDA/1OR0000.png"
    str_imgRawPathIn9=r"./COLAB_instance-level-human-parsing2/embryoDataset/video_animal2_proyeccion_SeqDA2/1OR0000.png"

    #MASK
    image = tf.io.read_file(str_imgMaskPathIn9)
    image = tf.image.decode_png(image, channels=1)
    image.set_shape([None, None, 1])
    image = tf.cast(image, tf.float32) 
    #image = tf.image.resize(images=image, size=[1024,1024], method="nearest")#20230610

    nda_2DXX_uint8_imgMask = image.numpy()#(1024, 1024, 1)
    nda_2DXX_uint8_imgMask = np.squeeze(nda_2DXX_uint8_imgMask)

    #RAW
    image = tf.io.read_file(str_imgRawPathIn9)
    image = tf.image.decode_png(image, channels=3)
    image.set_shape([None, None, 3])
    nda_3DXX3_uint8_imgRGB = image.numpy()
    #image = tf.image.resize(images=image, size=[1024,1024])#20230610


    int_valorDiscontinuidadIn_Raw=0
    str_franjaColor_Raw="unicolor_interseccion"
    lst_int_ColDisc1=None
    int_ColGrosor1=None
    lst_int_RowDisc1=[50]
    int_RowGrosor1=10


    nda_3DXX3_uint8_imgRGB_disc=CTI.func_deteriorarImagenDiscontinuidades(nda_2DXXo3DXX3_imgIn=nda_3DXX3_uint8_imgRGB, #20230616
                                                                                                        lst_int_RowDisc=lst_int_RowDisc1,
                                                                                                        int_RowGrosor=int_RowGrosor1,
                                                                                                        lst_int_ColDisc=None,
                                                                                                        int_ColGrosor=None,
                                                                                                        int_valorDiscontinuidadIn=int_valorDiscontinuidadIn_Raw,#None,
                                                                                                        #boo_normalizar0_255=True,
                                                                                                        #str_franjaColor="azarEntre_127_128_paraRGB_interseccion",
                                                                                                        str_franjaColor=str_franjaColor_Raw,#Para deeplab
                                                                                                        nda_2DXX_imgMaskIn=nda_2DXX_uint8_imgMask
                                                                                                        #nda_2DXX_imgMaskIn=None
                                                                                            )
    plt.imshow(nda_3DXX3_uint8_imgRGB_disc)
    plt.show()

    print("adass")

    Func_model=tf.keras.models.load_model(r"saved_models/20230709_0917_A20220911_class_DeepLabVBS_DatasetEmbrionBa3NC2Tr105Va10Te30Ep29D0_6Mi75_8")
    #Func_model=tf.keras.models.load_model(r"saved_models/"+str_dirModelLoadIn9) #20230119_1734
    print("adass2")
    #Func_model.predict(np.expand_dims(nda_3DXX3_uint8_imgRGB_disc, axis=0))#Loagregue 20290929
    print("nda_3DXX3_uint8_imgRGB.shape: ", nda_3DXX3_uint8_imgRGB.shape)

    #Func_model.predict(np.expand_dims((nda_3DXX3_uint8_imgRGB_disc / 127.5 - 1), axis=0))
    nda_3DXX2_probmapDL = Func_model.predict(np.expand_dims((nda_3DXX3_uint8_imgRGB_disc / 127.5 - 1), axis=0))#Estamos dentro de sourcestart, ConfiguracionesTipo2
    nda_3DXX2_probmapDL = np.squeeze(nda_3DXX2_probmapDL)

    nda_3DXX2_probmapDL_Argmax = np.argmax(nda_3DXX2_probmapDL, axis=2)

    plt.imshow(nda_3DXX2_probmapDL_Argmax)
    plt.show()
    print("te predije")

In [ ]:
if False:    
    import os
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
    import tensorflow as tf
    import numpy as np

    print("TensorFlow version:", tf.__version__)

    str_dirModelLoadIn9=r"20230709_0917_A20220911_class_DeepLabVBS_DatasetEmbrionBa3NC2Tr105Va10Te30Ep29D0_6Mi75_8"
    str_imgRawPathIn9=r"./COLAB_instance-level-human-parsing2/embryoDataset/video_animal2_proyeccion_SeqDA2/1OR0000.png"
    image = tf.io.read_file(str_imgRawPathIn9)
    image = tf.image.decode_png(image, channels=3)
    image.set_shape([None, None, 3])
    #image = tf.image.resize(images=image, size=[1024,1024])#20230610

    nda_3DXX3_uint8_imgRGB = image.numpy()
    Func_model=tf.keras.models.load_model(r"saved_models/"+str_dirModelLoadIn9) #20230119_1734
    print("basket")
    Func_model.predict(np.expand_dims((nda_3DXX3_uint8_imgRGB / 127.5 - 1), 
                               axis=0))#Loagregue 20290929
    print("futbol")


In [ ]:
#os.environ["CUDA_VISIBLE_DEVICES"]